# **Базовые регрессии (без эффектов, с тайм-эффектами, со страновыми и тайм-эффектами)**

In [2]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS

df = pd.read_excel("data5.xlsx", sheet_name=1, index_col='state_name', na_values='..')

# оставляем страны, по которым есть полные данные с 2003 по 2022
df = df.groupby('state_name').filter(lambda x: x['year'].min() == 2002 and x['year'].max() == 2022)

# удаляем чрезмерно экономически развитые страны (выбросы)
df = df.drop(['Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'Trinidad and Tobago', 
              'United Arab Emirates', 'Chile', 'Croatia', 'Hungary', 'Poland', 'Uruguay', 'Equatorial Guinea'])

# удаляем Океанию
df = df[df['region'] != 7]

In [3]:
# логарифмируем ВВП и FDI
df['log_gdppc'] = np.log(df['gdp_per_capita'])
df['log_fdipc'] = np.log(df['fdi_per_capita'])


# лагируем переменные
cols = ['libdem_norm_diff', 'property_rights_diff', 'tax_burden_diff', 'government_spending_diff', 
        'business_freedom_diff', 'monetary_freedom_diff', 'trade_freedom_diff', 'investment_freedom_diff', 
        'financial_freedom_diff', 'gov_effectiveness_diff', 'corruption_diff', 'pstab_diff']

for col in cols:
    df[f'{col}_lagged'] = df.groupby('state_name')[col].shift(-1)

In [4]:
# мультииндекс для регрессий (2022 исключён, т.к. для ряда нет лагированных зависимых переменных)
df = df[df['year'] != 2022].reset_index().set_index(['state_name', 'year'])

# добавляем в список независимую и контрольные переменные
independent_vars = df[['imf_pr', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab']]

## **Уровень демократии**

In [5]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,0.0468
No. Observations:,1740,R-squared (Within):,0.0017
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0034
Time:,17:02:15,Log-likelihood,-4879.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.1806
Entities:,87,P-value,0.3163
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1864


In [6]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0043
Estimator:,PanelOLS,R-squared (Between):,-12.528
No. Observations:,1740,R-squared (Within):,0.0013
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4702
Time:,17:02:15,Log-likelihood,-4865.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.4808
Entities:,87,P-value,0.1929
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.2510


In [7]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0069
Estimator:,PanelOLS,R-squared (Between):,-6.7346
No. Observations:,1740,R-squared (Within):,0.0069
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2468
Time:,17:02:15,Log-likelihood,-4843.2
Cov. Estimator:,Robust,,
,,F-statistic:,2.2735
Entities:,87,P-value,0.0451
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.7779


In [8]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0031
Estimator:,PanelOLS,R-squared (Between):,-27.939
No. Observations:,1740,R-squared (Within):,0.0061
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.0455
Time:,17:02:16,Log-likelihood,-4833.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.0185
Entities:,87,P-value,0.4051
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1721


## **Защита прав собственности**

In [9]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0088
Estimator:,PanelOLS,R-squared (Between):,0.1489
No. Observations:,1740,R-squared (Within):,0.0060
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0088
Time:,17:02:16,Log-likelihood,-5545.8
Cov. Estimator:,Robust,,
,,F-statistic:,3.0763
Entities:,87,P-value,0.0091
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.2647


In [10]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0040
Estimator:,PanelOLS,R-squared (Between):,-0.9055
No. Observations:,1740,R-squared (Within):,0.0008
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0168
Time:,17:02:16,Log-likelihood,-5159.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.3784
Entities:,87,P-value,0.2294
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2924


In [11]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0244
Estimator:,PanelOLS,R-squared (Between):,-46.690
No. Observations:,1740,R-squared (Within):,0.0244
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.8783
Time:,17:02:16,Log-likelihood,-5515.0
Cov. Estimator:,Robust,,
,,F-statistic:,8.2509
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,7.5411


In [12]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-1.0072
No. Observations:,1740,R-squared (Within):,-0.0121
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0314
Time:,17:02:16,Log-likelihood,-5138.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.5322
Entities:,87,P-value,0.7521
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5452


## **Налоговое бремя**

In [13]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0288
Estimator:,PanelOLS,R-squared (Between):,0.3559
No. Observations:,1740,R-squared (Within):,0.0093
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0288
Time:,17:02:16,Log-likelihood,-4407.0
Cov. Estimator:,Robust,,
,,F-statistic:,10.293
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,8.3674


In [14]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0054
Estimator:,PanelOLS,R-squared (Between):,-3.7590
No. Observations:,1740,R-squared (Within):,0.0091
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2032
Time:,17:02:16,Log-likelihood,-4378.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.8718
Entities:,87,P-value,0.0962
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6800


In [15]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0172
Estimator:,PanelOLS,R-squared (Between):,-40.065
No. Observations:,1740,R-squared (Within):,0.0172
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.2412
Time:,17:02:16,Log-likelihood,-4366.9
Cov. Estimator:,Robust,,
,,F-statistic:,5.7593
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.7761


In [16]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-8.6937
No. Observations:,1740,R-squared (Within):,0.0137
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4769
Time:,17:02:16,Log-likelihood,-4344.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.8351
Entities:,87,P-value,0.1029
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5223


## **Государственные расходы**

In [17]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0007
Estimator:,PanelOLS,R-squared (Between):,0.0005
No. Observations:,1740,R-squared (Within):,0.0007
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0007
Time:,17:02:16,Log-likelihood,-5549.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.2556
Entities:,87,P-value,0.9371
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2541


In [18]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0003
Estimator:,PanelOLS,R-squared (Between):,-0.0937
No. Observations:,1740,R-squared (Within):,6.865e-05
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0009
Time:,17:02:16,Log-likelihood,-5526.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.1125
Entities:,87,P-value,0.9896
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.1244


In [19]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0072
Estimator:,PanelOLS,R-squared (Between):,-28.541
No. Observations:,1740,R-squared (Within):,0.0072
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3008
Time:,17:02:16,Log-likelihood,-5534.5
Cov. Estimator:,Robust,,
,,F-statistic:,2.3889
Entities:,87,P-value,0.0360
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.1055


In [20]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-9.1212
No. Observations:,1740,R-squared (Within):,0.0040
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0944
Time:,17:02:16,Log-likelihood,-5515.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.5313
Entities:,87,P-value,0.7527
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5368


## **Свобода бизнеса**

In [21]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0052
Estimator:,PanelOLS,R-squared (Between):,0.2888
No. Observations:,1740,R-squared (Within):,-0.0001
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0052
Time:,17:02:16,Log-likelihood,-5307.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.8068
Entities:,87,P-value,0.1083
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4895


In [22]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0054
Estimator:,PanelOLS,R-squared (Between):,0.1287
No. Observations:,1740,R-squared (Within):,-0.0014
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0010
Time:,17:02:16,Log-likelihood,-5257.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.8491
Entities:,87,P-value,0.1003
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6502


In [23]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0042
Estimator:,PanelOLS,R-squared (Between):,-12.559
No. Observations:,1740,R-squared (Within):,0.0042
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2259
Time:,17:02:16,Log-likelihood,-5292.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.3855
Entities:,87,P-value,0.2267
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3101


In [24]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0024
Estimator:,PanelOLS,R-squared (Between):,-114.32
No. Observations:,1740,R-squared (Within):,-0.0230
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.1160
Time:,17:02:16,Log-likelihood,-5243.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.7735
Entities:,87,P-value,0.5688
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5892


## **Монетарная свобода**

In [25]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0047
Estimator:,PanelOLS,R-squared (Between):,0.0938
No. Observations:,1740,R-squared (Within):,0.0018
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0047
Time:,17:02:16,Log-likelihood,-5139.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.6274
Entities:,87,P-value,0.1495
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3893


In [26]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0047
Estimator:,PanelOLS,R-squared (Between):,-1.3137
No. Observations:,1740,R-squared (Within):,0.0017
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0396
Time:,17:02:16,Log-likelihood,-5080.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.6346
Entities:,87,P-value,0.1476
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5444


In [27]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,-0.1155
No. Observations:,1740,R-squared (Within):,0.0023
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0014
Time:,17:02:16,Log-likelihood,-5113.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.7499
Entities:,87,P-value,0.5862
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7372


In [28]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0020
Estimator:,PanelOLS,R-squared (Between):,-16.846
No. Observations:,1740,R-squared (Within):,9.935e-05
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5288
Time:,17:02:16,Log-likelihood,-5053.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.6494
Entities:,87,P-value,0.6620
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5850


## **Свобода торговли**

In [29]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0081
Estimator:,PanelOLS,R-squared (Between):,-0.0086
No. Observations:,1740,R-squared (Within):,0.0084
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0081
Time:,17:02:16,Log-likelihood,-5567.9
Cov. Estimator:,Robust,,
,,F-statistic:,2.8330
Entities:,87,P-value,0.0149
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7827


In [30]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0011
Estimator:,PanelOLS,R-squared (Between):,-0.0261
No. Observations:,1740,R-squared (Within):,0.0011
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0007
Time:,17:02:16,Log-likelihood,-5509.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.3894
Entities:,87,P-value,0.8564
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3552


In [31]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0198
Estimator:,PanelOLS,R-squared (Between):,-30.467
No. Observations:,1740,R-squared (Within):,0.0198
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5019
Time:,17:02:17,Log-likelihood,-5542.6
Cov. Estimator:,Robust,,
,,F-statistic:,6.6454
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.6780


In [32]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0019
Estimator:,PanelOLS,R-squared (Between):,-31.920
No. Observations:,1740,R-squared (Within):,-0.0127
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5587
Time:,17:02:17,Log-likelihood,-5495.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.6118
Entities:,87,P-value,0.6909
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3102


## **Свобода инвесторов**

In [33]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0022
Estimator:,PanelOLS,R-squared (Between):,-0.0264
No. Observations:,1740,R-squared (Within):,0.0030
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0022
Time:,17:02:17,Log-likelihood,-5576.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.7759
Entities:,87,P-value,0.5671
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0044


In [34]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0017
Estimator:,PanelOLS,R-squared (Between):,-0.6046
No. Observations:,1740,R-squared (Within):,-0.0018
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0181
Time:,17:02:17,Log-likelihood,-5514.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.5993
Entities:,87,P-value,0.7005
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5430


In [35]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0240
Estimator:,PanelOLS,R-squared (Between):,-297.30
No. Observations:,1740,R-squared (Within):,0.0240
Date:,"Mon, May 19 2025",R-squared (Overall):,-7.9904
Time:,17:02:17,Log-likelihood,-5533.8
Cov. Estimator:,Robust,,
,,F-statistic:,8.1032
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,5.5785


In [36]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0043
Estimator:,PanelOLS,R-squared (Between):,-19.630
No. Observations:,1740,R-squared (Within):,-0.0292
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5575
Time:,17:02:17,Log-likelihood,-5486.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.4011
Entities:,87,P-value,0.2209
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0006


## **Финансовая свобода**

In [37]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0049
Estimator:,PanelOLS,R-squared (Between):,0.0371
No. Observations:,1740,R-squared (Within):,0.0043
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0049
Time:,17:02:17,Log-likelihood,-5350.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.7123
Entities:,87,P-value,0.1285
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3801


In [38]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0039
Estimator:,PanelOLS,R-squared (Between):,-1.0146
No. Observations:,1740,R-squared (Within):,0.0041
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0150
Time:,17:02:17,Log-likelihood,-5305.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.3286
Entities:,87,P-value,0.2491
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0301


In [39]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0077
Estimator:,PanelOLS,R-squared (Between):,-4.2813
No. Observations:,1740,R-squared (Within):,0.0077
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0727
Time:,17:02:17,Log-likelihood,-5331.8
Cov. Estimator:,Robust,,
,,F-statistic:,2.5590
Entities:,87,P-value,0.0258
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6445


In [40]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,-9.3137
No. Observations:,1740,R-squared (Within):,0.0035
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1712
Time:,17:02:17,Log-likelihood,-5286.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.8866
Entities:,87,P-value,0.0936
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0789


## **Эффективность государственного управления**

In [41]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0009
Estimator:,PanelOLS,R-squared (Between):,0.0179
No. Observations:,1740,R-squared (Within):,0.0003
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0009
Time:,17:02:17,Log-likelihood,-5080.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.3192
Entities:,87,P-value,0.9017
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3136


In [42]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0014
Estimator:,PanelOLS,R-squared (Between):,-1.2766
No. Observations:,1740,R-squared (Within):,-0.0002
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0461
Time:,17:02:17,Log-likelihood,-5067.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.4688
Entities:,87,P-value,0.7997
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4358


In [43]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0028
Estimator:,PanelOLS,R-squared (Between):,-3.8065
No. Observations:,1740,R-squared (Within):,0.0028
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1341
Time:,17:02:17,Log-likelihood,-5046.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.9361
Entities:,87,P-value,0.4564
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8701


In [44]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0024
Estimator:,PanelOLS,R-squared (Between):,-1.2516
No. Observations:,1740,R-squared (Within):,0.0026
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0425
Time:,17:02:17,Log-likelihood,-5034.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.7864
Entities:,87,P-value,0.5594
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7254


## **Уровень коррупции**

In [45]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0044
Estimator:,PanelOLS,R-squared (Between):,0.0817
No. Observations:,1740,R-squared (Within):,0.0010
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0044
Time:,17:02:17,Log-likelihood,-4973.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.5164
Entities:,87,P-value,0.1815
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4401


In [46]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0042
Estimator:,PanelOLS,R-squared (Between):,-0.6677
No. Observations:,1740,R-squared (Within):,0.0007
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0269
Time:,17:02:17,Log-likelihood,-4958.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.4523
Entities:,87,P-value,0.2025
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4480


In [47]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0033
Estimator:,PanelOLS,R-squared (Between):,-3.3917
No. Observations:,1740,R-squared (Within):,0.0033
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1371
Time:,17:02:17,Log-likelihood,-4937.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.0889
Entities:,87,P-value,0.3645
Avg Obs:,20.000,Distribution:,"F(5,1648)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0149


In [48]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0031
Estimator:,PanelOLS,R-squared (Between):,-7.0491
No. Observations:,1740,R-squared (Within):,0.0032
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2884
Time:,17:02:17,Log-likelihood,-4922.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.0072
Entities:,87,P-value,0.4119
Avg Obs:,20.000,Distribution:,"F(5,1629)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8812


# **Регрессии с региональными эффектами**

In [49]:
# создаём дамми для проверки региональных эффектов
region_dummies = pd.get_dummies(df['region'], prefix='region', drop_first=True)

independent_vars_1 = pd.concat([independent_vars, region_dummies], axis=1)

## **Уровень демократии**

In [50]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0067
Estimator:,PanelOLS,R-squared (Between):,0.0984
No. Observations:,1740,R-squared (Within):,0.0031
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0067
Time:,17:02:17,Log-likelihood,-4876.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.1612
Entities:,87,P-value,0.3127
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6172


In [51]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0059
Estimator:,PanelOLS,R-squared (Between):,-11.040
No. Observations:,1740,R-squared (Within):,0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4131
Time:,17:02:17,Log-likelihood,-4864.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.0180
Entities:,87,P-value,0.4254
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3572


## **Защита прав собственности**

In [52]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0112
Estimator:,PanelOLS,R-squared (Between):,0.2266
No. Observations:,1740,R-squared (Within):,0.0069
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0112
Time:,17:02:17,Log-likelihood,-5543.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.9556
Entities:,87,P-value,0.0345
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3740


In [53]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0095
Estimator:,PanelOLS,R-squared (Between):,-2.3747
No. Observations:,1740,R-squared (Within):,-0.0031
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0489
Time:,17:02:17,Log-likelihood,-5154.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.6382
Entities:,87,P-value,0.0902
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5445


## **Налоговое бремя**

In [54]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0376
Estimator:,PanelOLS,R-squared (Between):,0.5018
No. Observations:,1740,R-squared (Within):,0.0099
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0376
Time:,17:02:17,Log-likelihood,-4399.1
Cov. Estimator:,Robust,,
,,F-statistic:,6.7534
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,5.3087


In [55]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0129
Estimator:,PanelOLS,R-squared (Between):,0.5097
No. Observations:,1740,R-squared (Within):,0.0067
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0350
Time:,17:02:18,Log-likelihood,-4372.1
Cov. Estimator:,Robust,,
,,F-statistic:,2.2401
Entities:,87,P-value,0.0136
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.7277


## **Государственные расходы**

In [56]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,0.0946
No. Observations:,1740,R-squared (Within):,0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0034
Time:,17:02:18,Log-likelihood,-5547.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.5967
Entities:,87,P-value,0.8177
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6685


In [57]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0021
Estimator:,PanelOLS,R-squared (Between):,-0.7413
No. Observations:,1740,R-squared (Within):,0.0009
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0071
Time:,17:02:18,Log-likelihood,-5524.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.3668
Entities:,87,P-value,0.9609
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3285


## **Свобода бизнеса**

In [58]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0102
Estimator:,PanelOLS,R-squared (Between):,0.4578
No. Observations:,1740,R-squared (Within):,0.0018
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0102
Time:,17:02:18,Log-likelihood,-5303.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.7765
Entities:,87,P-value,0.0600
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7094


In [59]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0092
Estimator:,PanelOLS,R-squared (Between):,-1.3501
No. Observations:,1740,R-squared (Within):,4.06e-05
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0247
Time:,17:02:18,Log-likelihood,-5254.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.5792
Entities:,87,P-value,0.1068
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3812


## **Монетарная свобода**

In [60]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0079
Estimator:,PanelOLS,R-squared (Between):,0.2054
No. Observations:,1740,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0079
Time:,17:02:18,Log-likelihood,-5136.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.3817
Entities:,87,P-value,0.1826
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2942


In [61]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0080
Estimator:,PanelOLS,R-squared (Between):,0.1472
No. Observations:,1740,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0060
Time:,17:02:18,Log-likelihood,-5078.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.3793
Entities:,87,P-value,0.1838
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3827


## **Свобода торговли**

In [62]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0103
Estimator:,PanelOLS,R-squared (Between):,0.0067
No. Observations:,1740,R-squared (Within):,0.0104
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0103
Time:,17:02:18,Log-likelihood,-5565.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.8067
Entities:,87,P-value,0.0547
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2860


In [63]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0018
Estimator:,PanelOLS,R-squared (Between):,-0.8664
No. Observations:,1740,R-squared (Within):,0.0014
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0135
Time:,17:02:18,Log-likelihood,-5509.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.3059
Entities:,87,P-value,0.9799
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2810


## **Свобода инвесторов**

In [64]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0039
Estimator:,PanelOLS,R-squared (Between):,0.0458
No. Observations:,1740,R-squared (Within):,0.0027
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0039
Time:,17:02:18,Log-likelihood,-5575.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.6774
Entities:,87,P-value,0.7463
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8354


In [65]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0044
Estimator:,PanelOLS,R-squared (Between):,-7.1265
No. Observations:,1740,R-squared (Within):,-0.0002
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1923
Time:,17:02:18,Log-likelihood,-5511.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.7615
Entities:,87,P-value,0.6663
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6267


## **Финансовая свобода**

In [66]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0074
Estimator:,PanelOLS,R-squared (Between):,0.1885
No. Observations:,1740,R-squared (Within):,0.0040
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0074
Time:,17:02:18,Log-likelihood,-5348.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.2941
Entities:,87,P-value,0.2280
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0803


In [67]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0067
Estimator:,PanelOLS,R-squared (Between):,-2.9842
No. Observations:,1740,R-squared (Within):,0.0032
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0528
Time:,17:02:18,Log-likelihood,-5303.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.1498
Entities:,87,P-value,0.3209
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0323


## **Эффективность государственного управления**

In [68]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,0.1130
No. Observations:,1740,R-squared (Within):,0.0006
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0046
Time:,17:02:18,Log-likelihood,-5077.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.7992
Entities:,87,P-value,0.6296
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6901


In [69]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0047
Estimator:,PanelOLS,R-squared (Between):,-0.1988
No. Observations:,1740,R-squared (Within):,0.0008
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0064
Time:,17:02:18,Log-likelihood,-5065.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.8069
Entities:,87,P-value,0.6221
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6733


## **Уровень коррупции**

In [70]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0102
Estimator:,PanelOLS,R-squared (Between):,0.2145
No. Observations:,1740,R-squared (Within):,0.0014
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0102
Time:,17:02:18,Log-likelihood,-4967.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.7833
Entities:,87,P-value,0.0587
Avg Obs:,20.000,Distribution:,"F(10,1730)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6770


In [71]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_1, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0105
Estimator:,PanelOLS,R-squared (Between):,-2.3802
No. Observations:,1740,R-squared (Within):,0.0017
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0968
Time:,17:02:18,Log-likelihood,-4952.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.8061
Entities:,87,P-value,0.0548
Avg Obs:,20.000,Distribution:,"F(10,1710)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7440


# **Базовые регрессии для периода 2003-2012 гг.**

In [72]:
df_2003_2012 = df.query('2003 <= year <= 2012')

independent_vars_2 = df_2003_2012[['imf_pr', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab']]

## **Уровень демократии**

In [73]:
dependent_var = df_2003_2012['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0004
Estimator:,PanelOLS,R-squared (Between):,0.0054
No. Observations:,870,R-squared (Within):,-0.0001
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0004
Time:,17:02:18,Log-likelihood,-2338.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.0768
Entities:,87,P-value,0.9957
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.1651


In [74]:
dependent_var = df_2003_2012['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0027
Estimator:,PanelOLS,R-squared (Between):,-3.5655
No. Observations:,870,R-squared (Within):,-0.0003
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3788
Time:,17:02:18,Log-likelihood,-2332.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.4626
Entities:,87,P-value,0.8042
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.9656


In [75]:
dependent_var = df_2003_2012['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0020
Estimator:,PanelOLS,R-squared (Between):,-1.8119
No. Observations:,870,R-squared (Within):,0.0020
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1905
Time:,17:02:18,Log-likelihood,-2289.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.3153
Entities:,87,P-value,0.9039
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.3199


In [76]:
dependent_var = df_2003_2012['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0036
Estimator:,PanelOLS,R-squared (Between):,-59.428
No. Observations:,870,R-squared (Within):,-0.0129
Date:,"Mon, May 19 2025",R-squared (Overall):,-6.3203
Time:,17:02:18,Log-likelihood,-2283.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.5529
Entities:,87,P-value,0.7362
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.3536


## **Защита прав собственности**

In [77]:
dependent_var = df_2003_2012['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0245
Estimator:,PanelOLS,R-squared (Between):,0.2193
No. Observations:,870,R-squared (Within):,-0.0001
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0245
Time:,17:02:18,Log-likelihood,-2354.8
Cov. Estimator:,Robust,,
,,F-statistic:,4.3528
Entities:,87,P-value,0.0006
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,4.8241


In [78]:
dependent_var = df_2003_2012['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0035
Estimator:,PanelOLS,R-squared (Between):,-1.5354
No. Observations:,870,R-squared (Within):,0.0008
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1718
Time:,17:02:18,Log-likelihood,-2345.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.5995
Entities:,87,P-value,0.7004
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.7446


In [79]:
dependent_var = df_2003_2012['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0083
Estimator:,PanelOLS,R-squared (Between):,-16.469
No. Observations:,870,R-squared (Within):,0.0083
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.8425
Time:,17:02:19,Log-likelihood,-2310.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.3091
Entities:,87,P-value,0.2579
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.5886


In [80]:
dependent_var = df_2003_2012['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0037
Estimator:,PanelOLS,R-squared (Between):,-21.630
No. Observations:,870,R-squared (Within):,-0.0210
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.4483
Time:,17:02:19,Log-likelihood,-2303.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.5657
Entities:,87,P-value,0.7264
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.4311


## **Налоговое бремя**

In [81]:
dependent_var = df_2003_2012['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0436
Estimator:,PanelOLS,R-squared (Between):,0.3444
No. Observations:,870,R-squared (Within):,0.0092
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0436
Time:,17:02:19,Log-likelihood,-2289.4
Cov. Estimator:,Robust,,
,,F-statistic:,7.8879
Entities:,87,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,7.4083


In [82]:
dependent_var = df_2003_2012['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0078
Estimator:,PanelOLS,R-squared (Between):,-0.7448
No. Observations:,870,R-squared (Within):,0.0092
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0681
Time:,17:02:19,Log-likelihood,-2281.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.3486
Entities:,87,P-value,0.2416
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.3233


In [83]:
dependent_var = df_2003_2012['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0151
Estimator:,PanelOLS,R-squared (Between):,-10.957
No. Observations:,870,R-squared (Within):,0.0151
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.1103
Time:,17:02:19,Log-likelihood,-2255.1
Cov. Estimator:,Robust,,
,,F-statistic:,2.3868
Entities:,87,P-value,0.0367
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,2.2073


In [84]:
dependent_var = df_2003_2012['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0092
Estimator:,PanelOLS,R-squared (Between):,-5.4896
No. Observations:,870,R-squared (Within):,0.0094
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5547
Time:,17:02:19,Log-likelihood,-2248.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.4343
Entities:,87,P-value,0.2096
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.4309


## **Государственные расходы**

In [85]:
dependent_var = df_2003_2012['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0079
Estimator:,PanelOLS,R-squared (Between):,0.1523
No. Observations:,870,R-squared (Within):,-0.0009
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0079
Time:,17:02:19,Log-likelihood,-2853.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.3859
Entities:,87,P-value,0.2271
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.3577


In [86]:
dependent_var = df_2003_2012['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0033
Estimator:,PanelOLS,R-squared (Between):,-4.0986
No. Observations:,870,R-squared (Within):,-0.0013
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2386
Time:,17:02:19,Log-likelihood,-2848.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.5654
Entities:,87,P-value,0.7266
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.6915


In [87]:
dependent_var = df_2003_2012['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0080
Estimator:,PanelOLS,R-squared (Between):,-4.8240
No. Observations:,870,R-squared (Within):,0.0080
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2718
Time:,17:02:19,Log-likelihood,-2827.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.2552
Entities:,87,P-value,0.2815
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.1608


In [88]:
dependent_var = df_2003_2012['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0119
Estimator:,PanelOLS,R-squared (Between):,-405.46
No. Observations:,870,R-squared (Within):,-0.0513
Date:,"Mon, May 19 2025",R-squared (Overall):,-23.526
Time:,17:02:19,Log-likelihood,-2821.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.8599
Entities:,87,P-value,0.0991
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.2719


## **Свобода бизнеса**

In [89]:
dependent_var = df_2003_2012['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0244
Estimator:,PanelOLS,R-squared (Between):,0.3064
No. Observations:,870,R-squared (Within):,-0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0244
Time:,17:02:19,Log-likelihood,-2665.7
Cov. Estimator:,Robust,,
,,F-statistic:,4.3193
Entities:,87,P-value,0.0007
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,3.9178


In [90]:
dependent_var = df_2003_2012['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0263
Estimator:,PanelOLS,R-squared (Between):,-4.1493
No. Observations:,870,R-squared (Within):,-0.0028
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3616
Time:,17:02:19,Log-likelihood,-2651.7
Cov. Estimator:,Robust,,
,,F-statistic:,4.6240
Entities:,87,P-value,0.0004
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,3.4873


In [91]:
dependent_var = df_2003_2012['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0108
Estimator:,PanelOLS,R-squared (Between):,-5.4719
No. Observations:,870,R-squared (Within):,0.0108
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4637
Time:,17:02:19,Log-likelihood,-2632.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.6951
Entities:,87,P-value,0.1333
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.8013


In [92]:
dependent_var = df_2003_2012['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0101
Estimator:,PanelOLS,R-squared (Between):,-17.671
No. Observations:,870,R-squared (Within):,0.0090
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.5210
Time:,17:02:19,Log-likelihood,-2619.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.5622
Entities:,87,P-value,0.1684
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.8914


## **Монетарная свобода**

In [93]:
dependent_var = df_2003_2012['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0099
Estimator:,PanelOLS,R-squared (Between):,0.1022
No. Observations:,870,R-squared (Within):,0.0037
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0099
Time:,17:02:19,Log-likelihood,-2526.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.7243
Entities:,87,P-value,0.1264
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.7041


In [94]:
dependent_var = df_2003_2012['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0106
Estimator:,PanelOLS,R-squared (Between):,-0.7430
No. Observations:,870,R-squared (Within):,0.0036
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0434
Time:,17:02:19,Log-likelihood,-2465.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.8402
Entities:,87,P-value,0.1026
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.5033


In [95]:
dependent_var = df_2003_2012['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0090
Estimator:,PanelOLS,R-squared (Between):,-4.2985
No. Observations:,870,R-squared (Within):,0.0090
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2622
Time:,17:02:19,Log-likelihood,-2498.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.4078
Entities:,87,P-value,0.2191
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.1125


In [96]:
dependent_var = df_2003_2012['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0117
Estimator:,PanelOLS,R-squared (Between):,-137.77
No. Observations:,870,R-squared (Within):,-0.0193
Date:,"Mon, May 19 2025",R-squared (Overall):,-8.6911
Time:,17:02:19,Log-likelihood,-2433.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.8282
Entities:,87,P-value,0.1050
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.4926


## **Свобода торговли**

In [97]:
dependent_var = df_2003_2012['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0395
Estimator:,PanelOLS,R-squared (Between):,0.4969
No. Observations:,870,R-squared (Within):,-0.0001
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0395
Time:,17:02:19,Log-likelihood,-2810.1
Cov. Estimator:,Robust,,
,,F-statistic:,7.1074
Entities:,87,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,8.2289


In [98]:
dependent_var = df_2003_2012['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0005
Estimator:,PanelOLS,R-squared (Between):,-0.2018
No. Observations:,870,R-squared (Within):,-0.0004
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0164
Time:,17:02:19,Log-likelihood,-2791.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.0819
Entities:,87,P-value,0.9951
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.0844


In [99]:
dependent_var = df_2003_2012['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0122
Estimator:,PanelOLS,R-squared (Between):,-41.827
No. Observations:,870,R-squared (Within):,0.0122
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.3222
Time:,17:02:19,Log-likelihood,-2786.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.9202
Entities:,87,P-value,0.0887
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.7369


In [100]:
dependent_var = df_2003_2012['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-0.5808
No. Observations:,870,R-squared (Within):,-0.0031
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0492
Time:,17:02:19,Log-likelihood,-2771.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.2437
Entities:,87,P-value,0.9430
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.2477


## **Свобода инвесторов**

In [101]:
dependent_var = df_2003_2012['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0073
Estimator:,PanelOLS,R-squared (Between):,0.0821
No. Observations:,870,R-squared (Within):,0.0012
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0073
Time:,17:02:19,Log-likelihood,-2895.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.2806
Entities:,87,P-value,0.2701
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.2306


In [102]:
dependent_var = df_2003_2012['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0105
Estimator:,PanelOLS,R-squared (Between):,-0.5362
No. Observations:,870,R-squared (Within):,-0.0037
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0441
Time:,17:02:19,Log-likelihood,-2866.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.8095
Entities:,87,P-value,0.1084
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.7573


In [103]:
dependent_var = df_2003_2012['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0286
Estimator:,PanelOLS,R-squared (Between):,-136.62
No. Observations:,870,R-squared (Within):,0.0286
Date:,"Mon, May 19 2025",R-squared (Overall):,-10.349
Time:,17:02:19,Log-likelihood,-2851.7
Cov. Estimator:,Robust,,
,,F-statistic:,4.5732
Entities:,87,P-value,0.0004
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,4.2401


In [104]:
dependent_var = df_2003_2012['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0071
Estimator:,PanelOLS,R-squared (Between):,-76.910
No. Observations:,870,R-squared (Within):,-0.0689
Date:,"Mon, May 19 2025",R-squared (Overall):,-5.9042
Time:,17:02:20,Log-likelihood,-2831.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.1070
Entities:,87,P-value,0.3551
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.1507


## **Финансовая свобода**

In [105]:
dependent_var = df_2003_2012['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0147
Estimator:,PanelOLS,R-squared (Between):,0.2061
No. Observations:,870,R-squared (Within):,0.0046
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0147
Time:,17:02:20,Log-likelihood,-2812.2
Cov. Estimator:,Robust,,
,,F-statistic:,2.5835
Entities:,87,P-value,0.0249
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,2.4342


In [106]:
dependent_var = df_2003_2012['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0138
Estimator:,PanelOLS,R-squared (Between):,-0.5019
No. Observations:,870,R-squared (Within):,0.0043
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0210
Time:,17:02:20,Log-likelihood,-2792.2
Cov. Estimator:,Robust,,
,,F-statistic:,2.3912
Entities:,87,P-value,0.0363
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,2.0731


In [107]:
dependent_var = df_2003_2012['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0100
Estimator:,PanelOLS,R-squared (Between):,-16.426
No. Observations:,870,R-squared (Within):,0.0100
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.8118
Time:,17:02:20,Log-likelihood,-2792.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.5655
Entities:,87,P-value,0.1674
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.5718


In [108]:
dependent_var = df_2003_2012['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0111
Estimator:,PanelOLS,R-squared (Between):,-98.489
No. Observations:,870,R-squared (Within):,-0.0158
Date:,"Mon, May 19 2025",R-squared (Overall):,-4.9394
Time:,17:02:20,Log-likelihood,-2771.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.7198
Entities:,87,P-value,0.1276
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.7106


## **Эффективность государственного управления**

In [109]:
dependent_var = df_2003_2012['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,0.0829
No. Observations:,870,R-squared (Within):,0.0004
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0058
Time:,17:02:20,Log-likelihood,-2514.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.0160
Entities:,87,P-value,0.4069
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.9715


In [110]:
dependent_var = df_2003_2012['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0065
Estimator:,PanelOLS,R-squared (Between):,0.0591
No. Observations:,870,R-squared (Within):,-0.0002
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0037
Time:,17:02:20,Log-likelihood,-2507.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.1115
Entities:,87,P-value,0.3526
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.0411


In [111]:
dependent_var = df_2003_2012['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0048
Estimator:,PanelOLS,R-squared (Between):,-27.903
No. Observations:,870,R-squared (Within):,0.0048
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.8266
Time:,17:02:20,Log-likelihood,-2485.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.7557
Entities:,87,P-value,0.5820
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.5662


In [112]:
dependent_var = df_2003_2012['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0101
Estimator:,PanelOLS,R-squared (Between):,-312.19
No. Observations:,870,R-squared (Within):,-0.0986
Date:,"Mon, May 19 2025",R-squared (Overall):,-20.580
Time:,17:02:20,Log-likelihood,-2475.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.5699
Entities:,87,P-value,0.1662
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.1850


## **Уровень коррупции**

In [113]:
dependent_var = df_2003_2012['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0031
Estimator:,PanelOLS,R-squared (Between):,0.0089
No. Observations:,870,R-squared (Within):,0.0026
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0031
Time:,17:02:20,Log-likelihood,-2513.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.5341
Entities:,87,P-value,0.7505
Avg Obs:,10.0000,Distribution:,"F(5,865)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.5289


In [114]:
dependent_var = df_2003_2012['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,-0.0600
No. Observations:,870,R-squared (Within):,0.0020
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0025
Time:,17:02:20,Log-likelihood,-2503.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.5836
Entities:,87,P-value,0.7126
Avg Obs:,10.0000,Distribution:,"F(5,855)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,0.6027


In [115]:
dependent_var = df_2003_2012['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0137
Estimator:,PanelOLS,R-squared (Between):,-42.327
No. Observations:,870,R-squared (Within):,0.0137
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.0189
Time:,17:02:20,Log-likelihood,-2476.6
Cov. Estimator:,Robust,,
,,F-statistic:,2.1613
Entities:,87,P-value,0.0565
Avg Obs:,10.0000,Distribution:,"F(5,778)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,2.0601


In [116]:
dependent_var = df_2003_2012['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_2, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0097
Estimator:,PanelOLS,R-squared (Between):,-23.476
No. Observations:,870,R-squared (Within):,0.0128
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.6695
Time:,17:02:20,Log-likelihood,-2468.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.5125
Entities:,87,P-value,0.1835
Avg Obs:,10.0000,Distribution:,"F(5,769)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,1.4673


# **Регрессии с учётом объёма структурных требований**

In [117]:
df_2003_2020 = df.query('2003 <= year <= 2020')

independent_vars_3 = df_2003_2020[['imf_pr', 'structural_conditions', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab']]

## **Уровень демократии**

In [118]:
dependent_var = df_2003_2020['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0033
Estimator:,PanelOLS,R-squared (Between):,0.0183
No. Observations:,1566,R-squared (Within):,0.0026
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0033
Time:,17:02:20,Log-likelihood,-4391.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.8563
Entities:,87,P-value,0.5265
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.0348


In [119]:
dependent_var = df_2003_2020['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-12.447
No. Observations:,1566,R-squared (Within):,0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5676
Time:,17:02:20,Log-likelihood,-4378.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.4461
Entities:,87,P-value,0.1934
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.0462


In [120]:
dependent_var = df_2003_2020['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0096
Estimator:,PanelOLS,R-squared (Between):,-6.4842
No. Observations:,1566,R-squared (Within):,0.0096
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2877
Time:,17:02:20,Log-likelihood,-4349.6
Cov. Estimator:,Robust,,
,,F-statistic:,2.3864
Entities:,87,P-value,0.0268
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.6801


In [121]:
dependent_var = df_2003_2020['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0074
Estimator:,PanelOLS,R-squared (Between):,-48.227
No. Observations:,1566,R-squared (Within):,0.0071
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.2015
Time:,17:02:20,Log-likelihood,-4340.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.8122
Entities:,87,P-value,0.0932
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.5010


## **Защита прав собственности**

In [122]:
dependent_var = df_2003_2020['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0206
Estimator:,PanelOLS,R-squared (Between):,0.3113
No. Observations:,1566,R-squared (Within):,0.0115
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0206
Time:,17:02:20,Log-likelihood,-4930.3
Cov. Estimator:,Robust,,
,,F-statistic:,5.4618
Entities:,87,P-value,0.0000
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,4.3057


In [123]:
dependent_var = df_2003_2020['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0075
Estimator:,PanelOLS,R-squared (Between):,0.1748
No. Observations:,1566,R-squared (Within):,0.0012
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0065
Time:,17:02:20,Log-likelihood,-4523.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.9522
Entities:,87,P-value,0.0694
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.8842


In [124]:
dependent_var = df_2003_2020['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0455
Estimator:,PanelOLS,R-squared (Between):,-84.707
No. Observations:,1566,R-squared (Within):,0.0455
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.5289
Time:,17:02:20,Log-likelihood,-4886.0
Cov. Estimator:,Robust,,
,,F-statistic:,11.692
Entities:,87,P-value,0.0000
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,11.192


In [125]:
dependent_var = df_2003_2020['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0035
Estimator:,PanelOLS,R-squared (Between):,-2.8353
No. Observations:,1566,R-squared (Within):,-0.0277
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1130
Time:,17:02:20,Log-likelihood,-4496.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.8522
Entities:,87,P-value,0.5296
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.8207


## **Налоговое бремя**

In [126]:
dependent_var = df_2003_2020['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0266
Estimator:,PanelOLS,R-squared (Between):,0.3705
No. Observations:,1566,R-squared (Within):,0.0084
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0266
Time:,17:02:20,Log-likelihood,-3915.8
Cov. Estimator:,Robust,,
,,F-statistic:,7.1072
Entities:,87,P-value,0.0000
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,5.6169


In [127]:
dependent_var = df_2003_2020['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0072
Estimator:,PanelOLS,R-squared (Between):,-0.9206
No. Observations:,1566,R-squared (Within):,0.0073
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0393
Time:,17:02:20,Log-likelihood,-3890.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.8578
Entities:,87,P-value,0.0847
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.6242


In [128]:
dependent_var = df_2003_2020['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0133
Estimator:,PanelOLS,R-squared (Between):,-24.447
No. Observations:,1566,R-squared (Within):,0.0133
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.2160
Time:,17:02:20,Log-likelihood,-3886.1
Cov. Estimator:,Robust,,
,,F-statistic:,3.3092
Entities:,87,P-value,0.0031
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.3889


In [129]:
dependent_var = df_2003_2020['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0054
Estimator:,PanelOLS,R-squared (Between):,-12.780
No. Observations:,1566,R-squared (Within):,-0.0025
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.6447
Time:,17:02:21,Log-likelihood,-3863.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.3124
Entities:,87,P-value,0.2483
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.0042


## **Государственные расходы**

In [130]:
dependent_var = df_2003_2020['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0027
Estimator:,PanelOLS,R-squared (Between):,0.0601
No. Observations:,1566,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0027
Time:,17:02:21,Log-likelihood,-4983.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.7062
Entities:,87,P-value,0.6447
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.6936


In [131]:
dependent_var = df_2003_2020['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0024
Estimator:,PanelOLS,R-squared (Between):,-5.2790
No. Observations:,1566,R-squared (Within):,0.0004
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1075
Time:,17:02:21,Log-likelihood,-4962.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.6187
Entities:,87,P-value,0.7155
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.5948


In [132]:
dependent_var = df_2003_2020['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-19.429
No. Observations:,1566,R-squared (Within):,0.0086
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3888
Time:,17:02:21,Log-likelihood,-4963.1
Cov. Estimator:,Robust,,
,,F-statistic:,2.1255
Entities:,87,P-value,0.0478
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.0067


In [133]:
dependent_var = df_2003_2020['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0026
Estimator:,PanelOLS,R-squared (Between):,-13.180
No. Observations:,1566,R-squared (Within):,-0.0007
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2701
Time:,17:02:21,Log-likelihood,-4945.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.6394
Entities:,87,P-value,0.6988
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.6394


## **Свобода бизнеса**

In [134]:
dependent_var = df_2003_2020['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0096
Estimator:,PanelOLS,R-squared (Between):,0.2721
No. Observations:,1566,R-squared (Within):,0.0004
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0096
Time:,17:02:21,Log-likelihood,-4616.1
Cov. Estimator:,Robust,,
,,F-statistic:,2.5176
Entities:,87,P-value,0.0199
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.5566


In [135]:
dependent_var = df_2003_2020['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0130
Estimator:,PanelOLS,R-squared (Between):,-12.527
No. Observations:,1566,R-squared (Within):,0.0011
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4215
Time:,17:02:21,Log-likelihood,-4593.8
Cov. Estimator:,Robust,,
,,F-statistic:,3.3935
Entities:,87,P-value,0.0025
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.5693


In [136]:
dependent_var = df_2003_2020['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,-13.385
No. Observations:,1566,R-squared (Within):,0.0034
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4482
Time:,17:02:21,Log-likelihood,-4594.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.8439
Entities:,87,P-value,0.5359
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.5309


In [137]:
dependent_var = df_2003_2020['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0055
Estimator:,PanelOLS,R-squared (Between):,-1.2292
No. Observations:,1566,R-squared (Within):,-0.0057
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0470
Time:,17:02:21,Log-likelihood,-4572.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.3539
Entities:,87,P-value,0.2300
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.0533


## **Монетарная свобода**

In [138]:
dependent_var = df_2003_2020['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0035
Estimator:,PanelOLS,R-squared (Between):,0.0658
No. Observations:,1566,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0035
Time:,17:02:21,Log-likelihood,-4646.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.9046
Entities:,87,P-value,0.4906
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.8353


In [139]:
dependent_var = df_2003_2020['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0045
Estimator:,PanelOLS,R-squared (Between):,-0.1670
No. Observations:,1566,R-squared (Within):,0.0013
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0037
Time:,17:02:21,Log-likelihood,-4590.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.1599
Entities:,87,P-value,0.3253
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.1201


In [140]:
dependent_var = df_2003_2020['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0031
Estimator:,PanelOLS,R-squared (Between):,-13.615
No. Observations:,1566,R-squared (Within):,0.0031
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4047
Time:,17:02:21,Log-likelihood,-4622.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.7563
Entities:,87,P-value,0.6045
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.4622


In [141]:
dependent_var = df_2003_2020['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0036
Estimator:,PanelOLS,R-squared (Between):,-5.1079
No. Observations:,1566,R-squared (Within):,-0.0006
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1535
Time:,17:02:21,Log-likelihood,-4565.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.8798
Entities:,87,P-value,0.5088
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.5892


## **Свобода торговли**

In [142]:
dependent_var = df_2003_2020['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0124
Estimator:,PanelOLS,R-squared (Between):,0.1280
No. Observations:,1566,R-squared (Within):,0.0092
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0124
Time:,17:02:21,Log-likelihood,-4883.1
Cov. Estimator:,Robust,,
,,F-statistic:,3.2532
Entities:,87,P-value,0.0035
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.4830


In [143]:
dependent_var = df_2003_2020['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0002
Estimator:,PanelOLS,R-squared (Between):,0.2686
No. Observations:,1566,R-squared (Within):,-0.0011
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0060
Time:,17:02:21,Log-likelihood,-4819.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.0628
Entities:,87,P-value,0.9990
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.0584


In [144]:
dependent_var = df_2003_2020['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0336
Estimator:,PanelOLS,R-squared (Between):,-129.69
No. Observations:,1566,R-squared (Within):,0.0336
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.4071
Time:,17:02:21,Log-likelihood,-4845.0
Cov. Estimator:,Robust,,
,,F-statistic:,8.5314
Entities:,87,P-value,0.0000
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,5.6717


In [145]:
dependent_var = df_2003_2020['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0003
Estimator:,PanelOLS,R-squared (Between):,-4.3576
No. Observations:,1566,R-squared (Within):,-0.0057
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1212
Time:,17:02:21,Log-likelihood,-4803.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.0637
Entities:,87,P-value,0.9990
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.0573


## **Свобода инвесторов**

In [146]:
dependent_var = df_2003_2020['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0061
Estimator:,PanelOLS,R-squared (Between):,0.0452
No. Observations:,1566,R-squared (Within):,0.0048
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0061
Time:,17:02:21,Log-likelihood,-4999.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.5981
Entities:,87,P-value,0.1439
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.8915


In [147]:
dependent_var = df_2003_2020['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0038
Estimator:,PanelOLS,R-squared (Between):,-0.9917
No. Observations:,1566,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0302
Time:,17:02:21,Log-likelihood,-4942.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.9807
Entities:,87,P-value,0.4367
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.8850


In [148]:
dependent_var = df_2003_2020['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0246
Estimator:,PanelOLS,R-squared (Between):,-295.23
No. Observations:,1566,R-squared (Within):,0.0246
Date:,"Mon, May 19 2025",R-squared (Overall):,-9.4063
Time:,17:02:21,Log-likelihood,-4959.4
Cov. Estimator:,Robust,,
,,F-statistic:,6.1950
Entities:,87,P-value,0.0000
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,4.0552


In [149]:
dependent_var = df_2003_2020['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0060
Estimator:,PanelOLS,R-squared (Between):,-21.035
No. Observations:,1566,R-squared (Within):,-0.0294
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7004
Time:,17:02:21,Log-likelihood,-4914.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.4696
Entities:,87,P-value,0.1850
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.1375


## **Финансовая свобода**

In [150]:
dependent_var = df_2003_2020['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0092
Estimator:,PanelOLS,R-squared (Between):,0.2064
No. Observations:,1566,R-squared (Within):,0.0028
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0092
Time:,17:02:21,Log-likelihood,-4676.0
Cov. Estimator:,Robust,,
,,F-statistic:,2.4183
Entities:,87,P-value,0.0249
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.2140


In [151]:
dependent_var = df_2003_2020['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0098
Estimator:,PanelOLS,R-squared (Between):,-2.2837
No. Observations:,1566,R-squared (Within):,0.0018
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0707
Time:,17:02:22,Log-likelihood,-4641.4
Cov. Estimator:,Robust,,
,,F-statistic:,2.5328
Entities:,87,P-value,0.0192
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,2.0618


In [152]:
dependent_var = df_2003_2020['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0060
Estimator:,PanelOLS,R-squared (Between):,-24.816
No. Observations:,1566,R-squared (Within):,0.0060
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7803
Time:,17:02:22,Log-likelihood,-4653.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.4877
Entities:,87,P-value,0.1786
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.1563


In [153]:
dependent_var = df_2003_2020['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0061
Estimator:,PanelOLS,R-squared (Between):,-56.580
No. Observations:,1566,R-squared (Within):,-0.0175
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.8095
Time:,17:02:22,Log-likelihood,-4618.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.4895
Entities:,87,P-value,0.1780
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.1033


## **Эффективность государственного управления**

In [154]:
dependent_var = df_2003_2020['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,0.0210
No. Observations:,1566,R-squared (Within):,0.0016
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0023
Time:,17:02:22,Log-likelihood,-4556.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.6040
Entities:,87,P-value,0.7273
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.6080


In [155]:
dependent_var = df_2003_2020['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0037
Estimator:,PanelOLS,R-squared (Between):,-2.2052
No. Observations:,1566,R-squared (Within):,0.0004
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0810
Time:,17:02:22,Log-likelihood,-4543.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.9484
Entities:,87,P-value,0.4591
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.9185


In [156]:
dependent_var = df_2003_2020['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0067
Estimator:,PanelOLS,R-squared (Between):,-18.861
No. Observations:,1566,R-squared (Within):,0.0067
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.6900
Time:,17:02:22,Log-likelihood,-4523.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.6436
Entities:,87,P-value,0.1314
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.3473


In [157]:
dependent_var = df_2003_2020['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0059
Estimator:,PanelOLS,R-squared (Between):,-8.3414
No. Observations:,1566,R-squared (Within):,0.0063
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3019
Time:,17:02:22,Log-likelihood,-4512.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.4350
Entities:,87,P-value,0.1976
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.1106


## **Уровень коррупции**

In [158]:
dependent_var = df_2003_2020['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0037
Estimator:,PanelOLS,R-squared (Between):,0.0455
No. Observations:,1566,R-squared (Within):,0.0020
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0037
Time:,17:02:22,Log-likelihood,-4428.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.9745
Entities:,87,P-value,0.4410
Avg Obs:,18.000,Distribution:,"F(6,1560)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,0.9267


In [159]:
dependent_var = df_2003_2020['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0051
Estimator:,PanelOLS,R-squared (Between):,-0.8467
No. Observations:,1566,R-squared (Within):,0.0006
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0336
Time:,17:02:22,Log-likelihood,-4414.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.3177
Entities:,87,P-value,0.2458
Avg Obs:,18.000,Distribution:,"F(6,1542)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.2047


In [160]:
dependent_var = df_2003_2020['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0087
Estimator:,PanelOLS,R-squared (Between):,-39.738
No. Observations:,1566,R-squared (Within):,0.0087
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.5976
Time:,17:02:22,Log-likelihood,-4392.5
Cov. Estimator:,Robust,,
,,F-statistic:,2.1570
Entities:,87,P-value,0.0446
Avg Obs:,18.000,Distribution:,"F(6,1473)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.7543


In [161]:
dependent_var = df_2003_2020['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_3, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0080
Estimator:,PanelOLS,R-squared (Between):,-24.263
No. Observations:,1566,R-squared (Within):,0.0082
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.9727
Time:,17:02:22,Log-likelihood,-4379.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.9518
Entities:,87,P-value,0.0695
Avg Obs:,18.000,Distribution:,"F(6,1456)"
Min Obs:,18.000,,
Max Obs:,18.000,F-statistic (robust):,1.5564


# **Модели со взаимодействиями**

In [162]:
# для проверки взаимодействий
df['imf_x_gdppc'] = df['imf_pr'] * df['log_gdppc']
df['imf_x_fdipc'] = df['imf_pr'] * df['log_fdipc']
df['imf_x_hdinorm'] = df['imf_pr'] * df['hdi_norm']

# новый список независимых переменных
independent_vars_4 = df[['imf_pr', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab', 'imf_x_gdppc', 'imf_x_fdipc', 'imf_x_hdinorm']]

## **Уровень демократии**

In [163]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0057
Estimator:,PanelOLS,R-squared (Between):,0.1134
No. Observations:,1740,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0057
Time:,17:02:22,Log-likelihood,-4877.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.2417
Entities:,87,P-value,0.2705
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2138


In [164]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0057
Estimator:,PanelOLS,R-squared (Between):,-17.469
No. Observations:,1740,R-squared (Within):,0.0022
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.6552
Time:,17:02:22,Log-likelihood,-4864.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.2340
Entities:,87,P-value,0.2749
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7574


In [165]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0075
Estimator:,PanelOLS,R-squared (Between):,-14.758
No. Observations:,1740,R-squared (Within):,0.0075
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5481
Time:,17:02:22,Log-likelihood,-4842.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.5606
Entities:,87,P-value,0.1318
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.8120


In [166]:
dependent_var = df['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0041
Estimator:,PanelOLS,R-squared (Between):,-42.878
No. Observations:,1740,R-squared (Within):,0.0068
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.6069
Time:,17:02:22,Log-likelihood,-4832.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.8433
Entities:,87,P-value,0.5644
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8803


## **Защита прав собственности**

In [167]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0093
Estimator:,PanelOLS,R-squared (Between):,0.1625
No. Observations:,1740,R-squared (Within):,0.0063
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0093
Time:,17:02:22,Log-likelihood,-5545.3
Cov. Estimator:,Robust,,
,,F-statistic:,2.0415
Entities:,87,P-value,0.0385
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5266


In [168]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0049
Estimator:,PanelOLS,R-squared (Between):,0.1446
No. Observations:,1740,R-squared (Within):,0.0002
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0030
Time:,17:02:22,Log-likelihood,-5158.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.0438
Entities:,87,P-value,0.4005
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0615


In [169]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0250
Estimator:,PanelOLS,R-squared (Between):,-65.070
No. Observations:,1740,R-squared (Within):,0.0250
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.2329
Time:,17:02:22,Log-likelihood,-5514.4
Cov. Estimator:,Robust,,
,,F-statistic:,5.2814
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,4.8874


In [170]:
dependent_var = df['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,-3.4909
No. Observations:,1740,R-squared (Within):,-0.0121
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0794
Time:,17:02:22,Log-likelihood,-5137.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.4714
Entities:,87,P-value,0.8770
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6407


## **Налоговое бремя**

In [171]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0312
Estimator:,PanelOLS,R-squared (Between):,0.3460
No. Observations:,1740,R-squared (Within):,0.0124
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0312
Time:,17:02:22,Log-likelihood,-4404.8
Cov. Estimator:,Robust,,
,,F-statistic:,6.9665
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,5.6450


In [172]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0055
Estimator:,PanelOLS,R-squared (Between):,-3.1291
No. Observations:,1740,R-squared (Within):,0.0093
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1676
Time:,17:02:22,Log-likelihood,-4378.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.1925
Entities:,87,P-value,0.2996
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0614


In [173]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0182
Estimator:,PanelOLS,R-squared (Between):,-21.688
No. Observations:,1740,R-squared (Within):,0.0182
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.2048
Time:,17:02:22,Log-likelihood,-4366.0
Cov. Estimator:,Robust,,
,,F-statistic:,3.8080
Entities:,87,P-value,0.0002
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.5457


In [174]:
dependent_var = df['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0065
Estimator:,PanelOLS,R-squared (Between):,-2.3631
No. Observations:,1740,R-squared (Within):,0.0151
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1188
Time:,17:02:23,Log-likelihood,-4343.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.3242
Entities:,87,P-value,0.2267
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0527


## **Государственные расходы**

In [175]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,0.0335
No. Observations:,1740,R-squared (Within):,0.0019
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0023
Time:,17:02:23,Log-likelihood,-5548.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.4917
Entities:,87,P-value,0.8629
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4058


In [176]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0037
Estimator:,PanelOLS,R-squared (Between):,-18.949
No. Observations:,1740,R-squared (Within):,0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2021
Time:,17:02:23,Log-likelihood,-5523.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.7880
Entities:,87,P-value,0.6133
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7219


In [177]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0104
Estimator:,PanelOLS,R-squared (Between):,-132.43
No. Observations:,1740,R-squared (Within):,0.0104
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.4186
Time:,17:02:23,Log-likelihood,-5531.7
Cov. Estimator:,Robust,,
,,F-statistic:,2.1650
Entities:,87,P-value,0.0275
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.1836


In [178]:
dependent_var = df['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0050
Estimator:,PanelOLS,R-squared (Between):,-16.827
No. Observations:,1740,R-squared (Within):,0.0071
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1745
Time:,17:02:23,Log-likelihood,-5512.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.0254
Entities:,87,P-value,0.4144
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9319


## **Свобода бизнеса**

In [179]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,0.2518
No. Observations:,1740,R-squared (Within):,0.0012
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0058
Time:,17:02:23,Log-likelihood,-5306.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.2603
Entities:,87,P-value,0.2601
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9980


In [180]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,-0.2785
No. Observations:,1740,R-squared (Within):,-1.927e-05
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0051
Time:,17:02:23,Log-likelihood,-5257.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.2420
Entities:,87,P-value,0.2703
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1173


In [181]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0064
Estimator:,PanelOLS,R-squared (Between):,-8.4372
No. Observations:,1740,R-squared (Within):,0.0064
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1482
Time:,17:02:23,Log-likelihood,-5290.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.3218
Entities:,87,P-value,0.2279
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4322


In [182]:
dependent_var = df['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,-193.50
No. Observations:,1740,R-squared (Within):,-0.0185
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.5615
Time:,17:02:23,Log-likelihood,-5241.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.9412
Entities:,87,P-value,0.4811
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7873


## **Монетарная свобода**

In [183]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0055
Estimator:,PanelOLS,R-squared (Between):,0.0742
No. Observations:,1740,R-squared (Within):,0.0033
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0055
Time:,17:02:23,Log-likelihood,-5138.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.1996
Entities:,87,P-value,0.2952
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1239


In [184]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0078
Estimator:,PanelOLS,R-squared (Between):,-14.011
No. Observations:,1740,R-squared (Within):,0.0037
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4363
Time:,17:02:23,Log-likelihood,-5078.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.6818
Entities:,87,P-value,0.0980
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7251


In [185]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0043
Estimator:,PanelOLS,R-squared (Between):,-7.7533
No. Observations:,1740,R-squared (Within):,0.0043
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2392
Time:,17:02:23,Log-likelihood,-5111.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.8981
Entities:,87,P-value,0.5171
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9234


In [186]:
dependent_var = df['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0057
Estimator:,PanelOLS,R-squared (Between):,-44.851
No. Observations:,1740,R-squared (Within):,0.0029
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.4053
Time:,17:02:23,Log-likelihood,-5050.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.1683
Entities:,87,P-value,0.3147
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1938


## **Свобода торговли**

In [187]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0094
Estimator:,PanelOLS,R-squared (Between):,-0.0483
No. Observations:,1740,R-squared (Within):,0.0104
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0094
Time:,17:02:23,Log-likelihood,-5566.8
Cov. Estimator:,Robust,,
,,F-statistic:,2.0505
Entities:,87,P-value,0.0376
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5939


In [188]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0025
Estimator:,PanelOLS,R-squared (Between):,0.1452
No. Observations:,1740,R-squared (Within):,0.0021
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0046
Time:,17:02:23,Log-likelihood,-5508.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.5311
Entities:,87,P-value,0.8338
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4809


In [189]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0224
Estimator:,PanelOLS,R-squared (Between):,-47.485
No. Observations:,1740,R-squared (Within):,0.0224
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7904
Time:,17:02:23,Log-likelihood,-5540.2
Cov. Estimator:,Robust,,
,,F-statistic:,4.7202
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.9367


In [190]:
dependent_var = df['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0038
Estimator:,PanelOLS,R-squared (Between):,-24.073
No. Observations:,1740,R-squared (Within):,-0.0102
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4220
Time:,17:02:23,Log-likelihood,-5493.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.7826
Entities:,87,P-value,0.6181
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5822


## **Свобода инвесторов**

In [191]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-0.0303
No. Observations:,1740,R-squared (Within):,0.0066
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0056
Time:,17:02:23,Log-likelihood,-5573.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.2135
Entities:,87,P-value,0.2869
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1436


In [192]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0018
Estimator:,PanelOLS,R-squared (Between):,-0.2483
No. Observations:,1740,R-squared (Within):,-0.0017
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0083
Time:,17:02:23,Log-likelihood,-5514.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.3785
Entities:,87,P-value,0.9324
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3443


In [193]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0243
Estimator:,PanelOLS,R-squared (Between):,-301.25
No. Observations:,1740,R-squared (Within):,0.0243
Date:,"Mon, May 19 2025",R-squared (Overall):,-8.0967
Time:,17:02:23,Log-likelihood,-5533.5
Cov. Estimator:,Robust,,
,,F-statistic:,5.1312
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.7190


In [194]:
dependent_var = df['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0049
Estimator:,PanelOLS,R-squared (Between):,-15.919
No. Observations:,1740,R-squared (Within):,-0.0306
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4589
Time:,17:02:24,Log-likelihood,-5485.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.9908
Entities:,87,P-value,0.4411
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7028


## **Финансовая свобода**

In [195]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0079
Estimator:,PanelOLS,R-squared (Between):,-0.0272
No. Observations:,1740,R-squared (Within):,0.0086
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0079
Time:,17:02:24,Log-likelihood,-5348.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.7280
Entities:,87,P-value,0.0873
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2116


In [196]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,-0.3504
No. Observations:,1740,R-squared (Within):,0.0081
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0014
Time:,17:02:24,Log-likelihood,-5304.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.2407
Entities:,87,P-value,0.2711
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8789


In [197]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0127
Estimator:,PanelOLS,R-squared (Between):,-17.489
No. Observations:,1740,R-squared (Within):,0.0127
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3155
Time:,17:02:24,Log-likelihood,-5327.5
Cov. Estimator:,Robust,,
,,F-statistic:,2.6398
Entities:,87,P-value,0.0071
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4154


In [198]:
dependent_var = df['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0101
Estimator:,PanelOLS,R-squared (Between):,-1.9672
No. Observations:,1740,R-squared (Within):,0.0076
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0295
Time:,17:02:24,Log-likelihood,-5283.1
Cov. Estimator:,Robust,,
,,F-statistic:,2.0698
Entities:,87,P-value,0.0357
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0403


## **Эффективность государственного управления**

In [199]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0012
Estimator:,PanelOLS,R-squared (Between):,0.0202
No. Observations:,1740,R-squared (Within):,0.0005
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0012
Time:,17:02:24,Log-likelihood,-5079.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.2653
Entities:,87,P-value,0.9770
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2763


In [200]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0019
Estimator:,PanelOLS,R-squared (Between):,-4.9635
No. Observations:,1740,R-squared (Within):,-0.0003
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1786
Time:,17:02:24,Log-likelihood,-5067.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.4030
Entities:,87,P-value,0.9193
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4248


In [201]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0029
Estimator:,PanelOLS,R-squared (Between):,-2.8422
No. Observations:,1740,R-squared (Within):,0.0029
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0993
Time:,17:02:24,Log-likelihood,-5046.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.6048
Entities:,87,P-value,0.7745
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5870


In [202]:
dependent_var = df['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0025
Estimator:,PanelOLS,R-squared (Between):,-1.3689
No. Observations:,1740,R-squared (Within):,0.0027
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0466
Time:,17:02:24,Log-likelihood,-5034.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.5063
Entities:,87,P-value,0.8523
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4864


## **Уровень коррупции**

In [203]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0069
Estimator:,PanelOLS,R-squared (Between):,0.1159
No. Observations:,1740,R-squared (Within):,0.0022
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0069
Time:,17:02:24,Log-likelihood,-4970.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.5106
Entities:,87,P-value,0.1484
Avg Obs:,20.000,Distribution:,"F(8,1732)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3945


In [204]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0066
Estimator:,PanelOLS,R-squared (Between):,0.0650
No. Observations:,1740,R-squared (Within):,0.0022
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0047
Time:,17:02:24,Log-likelihood,-4956.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.4274
Entities:,87,P-value,0.1800
Avg Obs:,20.000,Distribution:,"F(8,1712)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3467


In [205]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,-11.805
No. Observations:,1740,R-squared (Within):,0.0046
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4837
Time:,17:02:24,Log-likelihood,-4936.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.9418
Entities:,87,P-value,0.4806
Avg Obs:,20.000,Distribution:,"F(8,1645)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9146


In [206]:
dependent_var = df['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_4, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0043
Estimator:,PanelOLS,R-squared (Between):,-17.225
No. Observations:,1740,R-squared (Within):,0.0045
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7079
Time:,17:02:24,Log-likelihood,-4920.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.8874
Entities:,87,P-value,0.5263
Avg Obs:,20.000,Distribution:,"F(8,1626)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7993


# **Модели для стран с разным уровнем дохода**

In [207]:
# формируем значения
mean_gdppc_by_country = df.groupby('state_name')['gdp_per_capita'].mean()
percentile_33 = mean_gdppc_by_country.quantile(0.33)
percentile_66 = mean_gdppc_by_country.quantile(0.66)
# print(percentile_33.round(2), percentile_66.round(2)) 1298.81 4004.66

In [208]:
# делим страны на группы
low_gdp_countries = mean_gdppc_by_country[mean_gdppc_by_country <= percentile_33].index
middle_gdp_countries = mean_gdppc_by_country[(mean_gdppc_by_country > percentile_33) & (mean_gdppc_by_country <= percentile_66)].index
high_gdp_countries = mean_gdppc_by_country[mean_gdppc_by_country > percentile_66].index

# проверяем, что поделили правильно
# len(low_gdp_countries) + len(middle_gdp_countries) + len(high_gdp_countries) #87

## **Страны с низким уровнем дохода (средний ВВП на душу населения меньше 1292.81 $)**

In [209]:
df = df.reset_index()
df_low_gdp = df[df['state_name'].isin(low_gdp_countries)]
df = df.set_index(['state_name', 'year'])
df_low_gdp = df_low_gdp.set_index(['state_name', 'year'])


independent_vars_low_gdp = df_low_gdp[['imf_pr', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab']]

### **Уровень демократии**

In [210]:
dependent_var = df_low_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0049
Estimator:,PanelOLS,R-squared (Between):,0.0327
No. Observations:,580,R-squared (Within):,0.0042
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0049
Time:,17:02:24,Log-likelihood,-1699.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.5689
Entities:,29,P-value,0.7239
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7696


In [211]:
dependent_var = df_low_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0089
Estimator:,PanelOLS,R-squared (Between):,-60.213
No. Observations:,580,R-squared (Within):,0.0086
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.5724
Time:,17:02:24,Log-likelihood,-1687.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.0006
Entities:,29,P-value,0.4166
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3229


In [212]:
dependent_var = df_low_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0096
Estimator:,PanelOLS,R-squared (Between):,-49.391
No. Observations:,580,R-squared (Within):,0.0096
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.2873
Time:,17:02:24,Log-likelihood,-1690.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.0621
Entities:,29,P-value,0.3805
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2956


In [213]:
dependent_var = df_low_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0108
Estimator:,PanelOLS,R-squared (Between):,-370.12
No. Observations:,580,R-squared (Within):,-0.0155
Date:,"Mon, May 19 2025",R-squared (Overall):,-9.7320
Time:,17:02:24,Log-likelihood,-1678.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.1551
Entities:,29,P-value,0.3302
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2036


### **Защита прав собственности**

In [214]:
dependent_var = df_low_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0074
Estimator:,PanelOLS,R-squared (Between):,-0.2830
No. Observations:,580,R-squared (Within):,0.0120
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0074
Time:,17:02:24,Log-likelihood,-1783.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.8606
Entities:,29,P-value,0.5073
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6098


In [215]:
dependent_var = df_low_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,-1.6749
No. Observations:,580,R-squared (Within):,-0.0022
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0283
Time:,17:02:24,Log-likelihood,-1676.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.2531
Entities:,29,P-value,0.9383
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3962


In [216]:
dependent_var = df_low_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0277
Estimator:,PanelOLS,R-squared (Between):,-40.082
No. Observations:,580,R-squared (Within):,0.0277
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5972
Time:,17:02:24,Log-likelihood,-1773.3
Cov. Estimator:,Robust,,
,,F-statistic:,3.1077
Entities:,29,P-value,0.0089
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.8283


In [217]:
dependent_var = df_low_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0020
Estimator:,PanelOLS,R-squared (Between):,-2.1421
No. Observations:,580,R-squared (Within):,0.0128
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0208
Time:,17:02:25,Log-likelihood,-1670.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.2119
Entities:,29,P-value,0.9575
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.1555


### **Налоговое бремя**

In [218]:
dependent_var = df_low_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0221
Estimator:,PanelOLS,R-squared (Between):,0.2924
No. Observations:,580,R-squared (Within):,0.0114
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0221
Time:,17:02:25,Log-likelihood,-1481.0
Cov. Estimator:,Robust,,
,,F-statistic:,2.6047
Entities:,29,P-value,0.0242
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.7288


In [219]:
dependent_var = df_low_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0121
Estimator:,PanelOLS,R-squared (Between):,-17.146
No. Observations:,580,R-squared (Within):,0.0122
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.6427
Time:,17:02:25,Log-likelihood,-1464.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.3612
Entities:,29,P-value,0.2373
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4536


In [220]:
dependent_var = df_low_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0220
Estimator:,PanelOLS,R-squared (Between):,-30.649
No. Observations:,580,R-squared (Within):,0.0220
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.1487
Time:,17:02:25,Log-likelihood,-1469.8
Cov. Estimator:,Robust,,
,,F-statistic:,2.4521
Entities:,29,P-value,0.0327
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.6254


In [221]:
dependent_var = df_low_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0180
Estimator:,PanelOLS,R-squared (Between):,-11.798
No. Observations:,580,R-squared (Within):,0.0197
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4313
Time:,17:02:25,Log-likelihood,-1454.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.9304
Entities:,29,P-value,0.0877
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.0135


### **Государственные расходы**

In [222]:
dependent_var = df_low_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-0.1642
No. Observations:,580,R-squared (Within):,0.0030
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0016
Time:,17:02:25,Log-likelihood,-1830.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.1900
Entities:,29,P-value,0.9664
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2539


In [223]:
dependent_var = df_low_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0007
Estimator:,PanelOLS,R-squared (Between):,-0.8642
No. Observations:,580,R-squared (Within):,0.0012
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0056
Time:,17:02:25,Log-likelihood,-1821.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.0817
Entities:,29,P-value,0.9951
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.1151


In [224]:
dependent_var = df_low_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0294
Estimator:,PanelOLS,R-squared (Between):,-760.09
No. Observations:,580,R-squared (Within):,0.0294
Date:,"Mon, May 19 2025",R-squared (Overall):,-5.9778
Time:,17:02:25,Log-likelihood,-1819.5
Cov. Estimator:,Robust,,
,,F-statistic:,3.3025
Entities:,29,P-value,0.0060
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6911


In [225]:
dependent_var = df_low_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0265
Estimator:,PanelOLS,R-squared (Between):,-136.21
No. Observations:,580,R-squared (Within):,-0.0473
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.1234
Time:,17:02:25,Log-likelihood,-1811.5
Cov. Estimator:,Robust,,
,,F-statistic:,2.8659
Entities:,29,P-value,0.0145
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4605


### **Свобода бизнеса**

In [226]:
dependent_var = df_low_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0129
Estimator:,PanelOLS,R-squared (Between):,0.4354
No. Observations:,580,R-squared (Within):,0.0046
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0129
Time:,17:02:25,Log-likelihood,-1806.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.4985
Entities:,29,P-value,0.1884
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4889


In [227]:
dependent_var = df_low_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0109
Estimator:,PanelOLS,R-squared (Between):,-55.370
No. Observations:,580,R-squared (Within):,0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.0619
Time:,17:02:25,Log-likelihood,-1762.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.2289
Entities:,29,P-value,0.2941
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3329


In [228]:
dependent_var = df_low_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0129
Estimator:,PanelOLS,R-squared (Between):,-83.118
No. Observations:,580,R-squared (Within):,0.0129
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.5848
Time:,17:02:25,Log-likelihood,-1800.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.4326
Entities:,29,P-value,0.2107
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5800


In [229]:
dependent_var = df_low_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0116
Estimator:,PanelOLS,R-squared (Between):,-250.38
No. Observations:,580,R-squared (Within):,-0.0212
Date:,"Mon, May 19 2025",R-squared (Overall):,-4.8331
Time:,17:02:25,Log-likelihood,-1758.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.2321
Entities:,29,P-value,0.2926
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1126


### **Монетарная свобода**

In [230]:
dependent_var = df_low_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0044
Estimator:,PanelOLS,R-squared (Between):,0.3528
No. Observations:,580,R-squared (Within):,0.0008
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0044
Time:,17:02:25,Log-likelihood,-1844.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.5139
Entities:,29,P-value,0.7658
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6160


In [231]:
dependent_var = df_low_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0044
Estimator:,PanelOLS,R-squared (Between):,-0.4962
No. Observations:,580,R-squared (Within):,0.0012
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0040
Time:,17:02:25,Log-likelihood,-1827.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.4855
Entities:,29,P-value,0.7871
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6186


In [232]:
dependent_var = df_low_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0073
Estimator:,PanelOLS,R-squared (Between):,-188.83
No. Observations:,580,R-squared (Within):,0.0073
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.9560
Time:,17:02:25,Log-likelihood,-1840.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.7990
Entities:,29,P-value,0.5506
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7012


In [233]:
dependent_var = df_low_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,-76.421
No. Observations:,580,R-squared (Within):,0.0054
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7892
Time:,17:02:25,Log-likelihood,-1824.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.6149
Entities:,29,P-value,0.6886
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4764


### **Свобода торговли**

In [234]:
dependent_var = df_low_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0122
Estimator:,PanelOLS,R-squared (Between):,-0.2849
No. Observations:,580,R-squared (Within):,0.0167
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0122
Time:,17:02:25,Log-likelihood,-1932.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.4230
Entities:,29,P-value,0.2141
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2765


In [235]:
dependent_var = df_low_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0021
Estimator:,PanelOLS,R-squared (Between):,-6.1501
No. Observations:,580,R-squared (Within):,0.0104
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0818
Time:,17:02:25,Log-likelihood,-1919.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.2335
Entities:,29,P-value,0.9478
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3008


In [236]:
dependent_var = df_low_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0231
Estimator:,PanelOLS,R-squared (Between):,-2.0976
No. Observations:,580,R-squared (Within):,0.0231
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0087
Time:,17:02:25,Log-likelihood,-1924.5
Cov. Estimator:,Robust,,
,,F-statistic:,2.5789
Entities:,29,P-value,0.0255
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.0559


In [237]:
dependent_var = df_low_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0061
Estimator:,PanelOLS,R-squared (Between):,-10.107
No. Observations:,580,R-squared (Within):,0.0034
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1479
Time:,17:02:25,Log-likelihood,-1913.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.6466
Entities:,29,P-value,0.6642
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5452


### **Свобода инвесторов**

In [238]:
dependent_var = df_low_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0042
Estimator:,PanelOLS,R-squared (Between):,-0.0839
No. Observations:,580,R-squared (Within):,0.0057
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0042
Time:,17:02:25,Log-likelihood,-1858.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.4898
Entities:,29,P-value,0.7840
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7789


In [239]:
dependent_var = df_low_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-0.3984
No. Observations:,580,R-squared (Within):,0.0028
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0036
Time:,17:02:25,Log-likelihood,-1838.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.1734
Entities:,29,P-value,0.9725
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.1490


In [240]:
dependent_var = df_low_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0174
Estimator:,PanelOLS,R-squared (Between):,-14.041
No. Observations:,580,R-squared (Within):,0.0174
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2051
Time:,17:02:25,Log-likelihood,-1850.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.9295
Entities:,29,P-value,0.0877
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5396


In [241]:
dependent_var = df_low_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0137
Estimator:,PanelOLS,R-squared (Between):,-208.61
No. Observations:,580,R-squared (Within):,0.0038
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.2967
Time:,17:02:26,Log-likelihood,-1829.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.4641
Entities:,29,P-value,0.1999
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2499


### **Финансовая свобода**

In [242]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-0.0217
No. Observations:,580,R-squared (Within):,0.0058
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0056
Time:,17:02:26,Log-likelihood,-1777.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.6512
Entities:,29,P-value,0.6607
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7161


In [243]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0048
Estimator:,PanelOLS,R-squared (Between):,-3.4391
No. Observations:,580,R-squared (Within):,0.0053
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0177
Time:,17:02:26,Log-likelihood,-1748.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.5321
Entities:,29,P-value,0.7520
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4378


In [244]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0104
Estimator:,PanelOLS,R-squared (Between):,-42.835
No. Observations:,580,R-squared (Within):,0.0104
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2762
Time:,17:02:26,Log-likelihood,-1774.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.1461
Entities:,29,P-value,0.3348
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6858


In [245]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0104
Estimator:,PanelOLS,R-squared (Between):,-89.007
No. Observations:,580,R-squared (Within):,0.0073
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5880
Time:,17:02:26,Log-likelihood,-1744.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.1036
Entities:,29,P-value,0.3574
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6096


### **Эффективность государственного управления**

In [246]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-0.0217
No. Observations:,580,R-squared (Within):,0.0058
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0056
Time:,17:02:26,Log-likelihood,-1777.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.6512
Entities:,29,P-value,0.6607
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.7161


In [247]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0048
Estimator:,PanelOLS,R-squared (Between):,-3.4391
No. Observations:,580,R-squared (Within):,0.0053
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0177
Time:,17:02:26,Log-likelihood,-1748.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.5321
Entities:,29,P-value,0.7520
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4378


In [248]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0104
Estimator:,PanelOLS,R-squared (Between):,-42.835
No. Observations:,580,R-squared (Within):,0.0104
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2762
Time:,17:02:26,Log-likelihood,-1774.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.1461
Entities:,29,P-value,0.3348
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6858


In [249]:
dependent_var = df_low_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0104
Estimator:,PanelOLS,R-squared (Between):,-89.007
No. Observations:,580,R-squared (Within):,0.0073
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.5880
Time:,17:02:26,Log-likelihood,-1744.7
Cov. Estimator:,Robust,,
,,F-statistic:,1.1036
Entities:,29,P-value,0.3574
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6096


### **Уровень коррупции**

In [250]:
dependent_var = df_low_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0035
Estimator:,PanelOLS,R-squared (Between):,0.0612
No. Observations:,580,R-squared (Within):,0.0017
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0035
Time:,17:02:26,Log-likelihood,-1654.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.4048
Entities:,29,P-value,0.8456
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4156


In [251]:
dependent_var = df_low_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,-4.8659
No. Observations:,580,R-squared (Within):,0.0025
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1427
Time:,17:02:26,Log-likelihood,-1647.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.5138
Entities:,29,P-value,0.7659
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5404


In [252]:
dependent_var = df_low_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0048
Estimator:,PanelOLS,R-squared (Between):,-27.018
No. Observations:,580,R-squared (Within):,0.0048
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.8008
Time:,17:02:26,Log-likelihood,-1645.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.5271
Entities:,29,P-value,0.7558
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5354


In [253]:
dependent_var = df_low_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_low_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0041
Estimator:,PanelOLS,R-squared (Between):,-1.2280
No. Observations:,580,R-squared (Within):,-0.0051
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0416
Time:,17:02:26,Log-likelihood,-1638.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.4382
Entities:,29,P-value,0.8219
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4211


## **Страны со "средним" уровнем доходов (средний ВВП на душу населения в диапазоне от 1298.81 до 4004.66 $)**

In [254]:
df = df.reset_index()
df_middle_gdp = df[df['state_name'].isin(middle_gdp_countries)]
df = df.set_index(['state_name', 'year'])
df_middle_gdp = df_middle_gdp.set_index(['state_name', 'year'])


independent_vars_middle_gdp = df_middle_gdp[['imf_pr', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab']]

### **Уровень демократии**

In [255]:
dependent_var = df_middle_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0057
Estimator:,PanelOLS,R-squared (Between):,0.2276
No. Observations:,560,R-squared (Within):,-0.0033
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0057
Time:,17:02:26,Log-likelihood,-1601.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.6319
Entities:,28,P-value,0.6755
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1853


In [256]:
dependent_var = df_middle_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0099
Estimator:,PanelOLS,R-squared (Between):,-26.906
No. Observations:,560,R-squared (Within):,-0.0133
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.0590
Time:,17:02:26,Log-likelihood,-1590.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.0653
Entities:,28,P-value,0.3787
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5542


In [257]:
dependent_var = df_middle_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0073
Estimator:,PanelOLS,R-squared (Between):,-1.6087
No. Observations:,560,R-squared (Within):,0.0073
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0555
Time:,17:02:26,Log-likelihood,-1590.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.7776
Entities:,28,P-value,0.5661
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.0361


In [258]:
dependent_var = df_middle_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0061
Estimator:,PanelOLS,R-squared (Between):,-200.63
No. Observations:,560,R-squared (Within):,-0.0542
Date:,"Mon, May 19 2025",R-squared (Overall):,-7.8534
Time:,17:02:26,Log-likelihood,-1580.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.6281
Entities:,28,P-value,0.6784
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0279


### **Защита прав собственности**

In [259]:
dependent_var = df_middle_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0055
Estimator:,PanelOLS,R-squared (Between):,0.2363
No. Observations:,560,R-squared (Within):,0.0021
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0055
Time:,17:02:26,Log-likelihood,-1789.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.6096
Entities:,28,P-value,0.6926
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4646


In [260]:
dependent_var = df_middle_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0075
Estimator:,PanelOLS,R-squared (Between):,-23.689
No. Observations:,560,R-squared (Within):,-0.0008
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3448
Time:,17:02:26,Log-likelihood,-1671.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.8113
Entities:,28,P-value,0.5419
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8982


In [261]:
dependent_var = df_middle_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0273
Estimator:,PanelOLS,R-squared (Between):,-220.88
No. Observations:,560,R-squared (Within):,0.0273
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.1811
Time:,17:02:26,Log-likelihood,-1779.1
Cov. Estimator:,Robust,,
,,F-statistic:,2.9616
Entities:,28,P-value,0.0120
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.5364


In [262]:
dependent_var = df_middle_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0084
Estimator:,PanelOLS,R-squared (Between):,-17.439
No. Observations:,560,R-squared (Within):,-0.0084
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2615
Time:,17:02:26,Log-likelihood,-1666.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.8569
Entities:,28,P-value,0.5099
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5196


### **Налоговое бремя**

In [263]:
dependent_var = df_middle_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0445
Estimator:,PanelOLS,R-squared (Between):,0.5303
No. Observations:,560,R-squared (Within):,0.0109
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0445
Time:,17:02:26,Log-likelihood,-1466.0
Cov. Estimator:,Robust,,
,,F-statistic:,5.1707
Entities:,28,P-value,0.0001
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.9434


In [264]:
dependent_var = df_middle_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0098
Estimator:,PanelOLS,R-squared (Between):,-56.009
No. Observations:,560,R-squared (Within):,0.0256
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.5994
Time:,17:02:26,Log-likelihood,-1449.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.0620
Entities:,28,P-value,0.3805
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1343


In [265]:
dependent_var = df_middle_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0301
Estimator:,PanelOLS,R-squared (Between):,-119.43
No. Observations:,560,R-squared (Within):,0.0301
Date:,"Mon, May 19 2025",R-squared (Overall):,-7.6978
Time:,17:02:27,Log-likelihood,-1451.4
Cov. Estimator:,Robust,,
,,F-statistic:,3.2755
Entities:,28,P-value,0.0064
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.1997


In [266]:
dependent_var = df_middle_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0078
Estimator:,PanelOLS,R-squared (Between):,-239.55
No. Observations:,560,R-squared (Within):,0.0195
Date:,"Mon, May 19 2025",R-squared (Overall):,-15.478
Time:,17:02:27,Log-likelihood,-1439.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.8034
Entities:,28,P-value,0.5476
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5692


### **Государственные расходы**

In [341]:
dependent_var = df_middle_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0047
Estimator:,PanelOLS,R-squared (Between):,-0.2594
No. Observations:,560,R-squared (Within):,0.0065
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0047
Time:,17:07:04,Log-likelihood,-1744.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.5208
Entities:,28,P-value,0.7606
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4254


In [342]:
dependent_var = df_middle_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0038
Estimator:,PanelOLS,R-squared (Between):,-25.648
No. Observations:,560,R-squared (Within):,0.0051
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1684
Time:,17:07:07,Log-likelihood,-1730.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.4057
Entities:,28,P-value,0.8449
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3654


In [343]:
dependent_var = df_middle_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0119
Estimator:,PanelOLS,R-squared (Between):,-40.524
No. Observations:,560,R-squared (Within):,0.0119
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2622
Time:,17:07:10,Log-likelihood,-1740.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.2658
Entities:,28,P-value,0.2773
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5511


In [344]:
dependent_var = df_middle_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0090
Estimator:,PanelOLS,R-squared (Between):,-323.38
No. Observations:,560,R-squared (Within):,0.0054
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.1814
Time:,17:07:14,Log-likelihood,-1726.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.9252
Entities:,28,P-value,0.4642
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8531


### **Свобода бизнеса**

In [271]:
dependent_var = df_middle_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0061
Estimator:,PanelOLS,R-squared (Between):,0.2890
No. Observations:,560,R-squared (Within):,0.0015
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0061
Time:,17:02:27,Log-likelihood,-1735.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.6821
Entities:,28,P-value,0.6372
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6082


In [272]:
dependent_var = df_middle_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0119
Estimator:,PanelOLS,R-squared (Between):,-224.63
No. Observations:,560,R-squared (Within):,0.0065
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.6275
Time:,17:02:27,Log-likelihood,-1718.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.2875
Entities:,28,P-value,0.2677
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9583


In [273]:
dependent_var = df_middle_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0092
Estimator:,PanelOLS,R-squared (Between):,-168.91
No. Observations:,560,R-squared (Within):,0.0092
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.7236
Time:,17:02:27,Log-likelihood,-1729.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.9766
Entities:,28,P-value,0.4314
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8535


In [274]:
dependent_var = df_middle_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0074
Estimator:,PanelOLS,R-squared (Between):,-444.32
No. Observations:,560,R-squared (Within):,0.0048
Date:,"Mon, May 19 2025",R-squared (Overall):,-7.1834
Time:,17:02:27,Log-likelihood,-1715.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.7606
Entities:,28,P-value,0.5786
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5898


### **Монетарная свобода**

In [275]:
dependent_var = df_middle_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0008
Estimator:,PanelOLS,R-squared (Between):,0.0600
No. Observations:,560,R-squared (Within):,-0.0005
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0008
Time:,17:02:27,Log-likelihood,-1540.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.0853
Entities:,28,P-value,0.9945
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.0898


In [276]:
dependent_var = df_middle_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0077
Estimator:,PanelOLS,R-squared (Between):,-145.32
No. Observations:,560,R-squared (Within):,-0.0049
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.9649
Time:,17:02:27,Log-likelihood,-1495.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.8277
Entities:,28,P-value,0.5302
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6337


In [277]:
dependent_var = df_middle_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0014
Estimator:,PanelOLS,R-squared (Between):,-19.531
No. Observations:,560,R-squared (Within):,0.0014
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3965
Time:,17:02:27,Log-likelihood,-1534.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.1429
Entities:,28,P-value,0.9821
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.0935


In [278]:
dependent_var = df_middle_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0062
Estimator:,PanelOLS,R-squared (Between):,-530.30
No. Observations:,560,R-squared (Within):,-0.0460
Date:,"Mon, May 19 2025",R-squared (Overall):,-10.848
Time:,17:02:27,Log-likelihood,-1489.3
Cov. Estimator:,Robust,,
,,F-statistic:,0.6324
Entities:,28,P-value,0.6751
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5160


### **Свобода торговли**

In [279]:
dependent_var = df_middle_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0156
Estimator:,PanelOLS,R-squared (Between):,0.1458
No. Observations:,560,R-squared (Within):,0.0125
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0156
Time:,17:02:27,Log-likelihood,-1795.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.7574
Entities:,28,P-value,0.1198
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1111


In [280]:
dependent_var = df_middle_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0025
Estimator:,PanelOLS,R-squared (Between):,-4.7041
No. Observations:,560,R-squared (Within):,0.0073
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1010
Time:,17:02:27,Log-likelihood,-1766.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.2711
Entities:,28,P-value,0.9289
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2335


In [281]:
dependent_var = df_middle_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0226
Estimator:,PanelOLS,R-squared (Between):,-58.231
No. Observations:,560,R-squared (Within):,0.0226
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.3164
Time:,17:02:27,Log-likelihood,-1786.9
Cov. Estimator:,Robust,,
,,F-statistic:,2.4418
Entities:,28,P-value,0.0334
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7657


In [282]:
dependent_var = df_middle_gdp['trade_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0060
Estimator:,PanelOLS,R-squared (Between):,-4.1825
No. Observations:,560,R-squared (Within):,-0.0038
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0999
Time:,17:02:27,Log-likelihood,-1760.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.6173
Entities:,28,P-value,0.6867
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6625


### **Свобода инвесторов**

In [283]:
dependent_var = df_middle_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0022
Estimator:,PanelOLS,R-squared (Between):,-0.0894
No. Observations:,560,R-squared (Within):,0.0050
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0022
Time:,17:02:27,Log-likelihood,-1796.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.2494
Entities:,28,P-value,0.9401
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3386


In [284]:
dependent_var = df_middle_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0042
Estimator:,PanelOLS,R-squared (Between):,-0.2499
No. Observations:,560,R-squared (Within):,-0.0137
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0207
Time:,17:02:27,Log-likelihood,-1772.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.4501
Entities:,28,P-value,0.8133
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4066


In [285]:
dependent_var = df_middle_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0320
Estimator:,PanelOLS,R-squared (Between):,-286.92
No. Observations:,560,R-squared (Within):,0.0320
Date:,"Mon, May 19 2025",R-squared (Overall):,-8.4550
Time:,17:02:27,Log-likelihood,-1779.3
Cov. Estimator:,Robust,,
,,F-statistic:,3.4822
Entities:,28,P-value,0.0042
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.2981


In [286]:
dependent_var = df_middle_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0048
Estimator:,PanelOLS,R-squared (Between):,-25.224
No. Observations:,560,R-squared (Within):,-0.0412
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7860
Time:,17:02:27,Log-likelihood,-1763.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.4901
Entities:,28,P-value,0.7837
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4842


### **Финансовая свобода**

In [287]:
dependent_var = df_middle_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0115
Estimator:,PanelOLS,R-squared (Between):,0.0721
No. Observations:,560,R-squared (Within):,0.0095
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0115
Time:,17:02:27,Log-likelihood,-1674.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.2896
Entities:,28,P-value,0.2668
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1873


In [288]:
dependent_var = df_middle_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0142
Estimator:,PanelOLS,R-squared (Between):,-56.583
No. Observations:,560,R-squared (Within):,0.0086
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.8274
Time:,17:02:27,Log-likelihood,-1652.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.5452
Entities:,28,P-value,0.1740
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3717


In [289]:
dependent_var = df_middle_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0223
Estimator:,PanelOLS,R-squared (Between):,-12.252
No. Observations:,560,R-squared (Within):,0.0223
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3759
Time:,17:02:28,Log-likelihood,-1661.7
Cov. Estimator:,Robust,,
,,F-statistic:,2.4026
Entities:,28,P-value,0.0360
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5540


In [290]:
dependent_var = df_middle_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0278
Estimator:,PanelOLS,R-squared (Between):,-523.45
No. Observations:,560,R-squared (Within):,-0.0534
Date:,"Mon, May 19 2025",R-squared (Overall):,-17.034
Time:,17:02:28,Log-likelihood,-1638.9
Cov. Estimator:,Robust,,
,,F-statistic:,2.9078
Entities:,28,P-value,0.0134
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.9781


### **Эффективность государственного управления**

In [291]:
dependent_var = df_middle_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0060
Estimator:,PanelOLS,R-squared (Between):,0.0742
No. Observations:,560,R-squared (Within):,0.0036
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0060
Time:,17:02:28,Log-likelihood,-1656.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.6713
Entities:,28,P-value,0.6454
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6533


In [292]:
dependent_var = df_middle_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0065
Estimator:,PanelOLS,R-squared (Between):,0.0519
No. Observations:,560,R-squared (Within):,0.0035
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0052
Time:,17:02:28,Log-likelihood,-1644.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.7023
Entities:,28,P-value,0.6219
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6474


In [293]:
dependent_var = df_middle_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0100
Estimator:,PanelOLS,R-squared (Between):,-43.008
No. Observations:,560,R-squared (Within):,0.0100
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.4824
Time:,17:02:28,Log-likelihood,-1645.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.0629
Entities:,28,P-value,0.3800
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1749


In [294]:
dependent_var = df_middle_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0130
Estimator:,PanelOLS,R-squared (Between):,-281.64
No. Observations:,560,R-squared (Within):,-0.0179
Date:,"Mon, May 19 2025",R-squared (Overall):,-9.7877
Time:,17:02:28,Log-likelihood,-1632.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.3431
Entities:,28,P-value,0.2447
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6855


### **Уровень коррупции**

In [295]:
dependent_var = df_middle_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0130
Estimator:,PanelOLS,R-squared (Between):,0.2219
No. Observations:,560,R-squared (Within):,0.0014
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0130
Time:,17:02:28,Log-likelihood,-1639.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.4577
Entities:,28,P-value,0.2020
Avg Obs:,20.000,Distribution:,"F(5,555)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3581


In [296]:
dependent_var = df_middle_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0154
Estimator:,PanelOLS,R-squared (Between):,-9.1073
No. Observations:,560,R-squared (Within):,0.0033
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4731
Time:,17:02:28,Log-likelihood,-1628.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.6730
Entities:,28,P-value,0.1393
Avg Obs:,20.000,Distribution:,"F(5,535)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7743


In [297]:
dependent_var = df_middle_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0088
Estimator:,PanelOLS,R-squared (Between):,-26.164
No. Observations:,560,R-squared (Within):,0.0088
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.3599
Time:,17:02:28,Log-likelihood,-1625.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.9304
Entities:,28,P-value,0.4607
Avg Obs:,20.000,Distribution:,"F(5,527)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9645


In [298]:
dependent_var = df_middle_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_middle_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0109
Estimator:,PanelOLS,R-squared (Between):,-43.905
No. Observations:,560,R-squared (Within):,0.0060
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.2903
Time:,17:02:28,Log-likelihood,-1614.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.1238
Entities:,28,P-value,0.3466
Avg Obs:,20.000,Distribution:,"F(5,508)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0996


## **Страны с "высоким" уровнем доходов (средний ВВП на душу населения более 4004.66$)**

In [299]:
df = df.reset_index()
df_high_gdp = df[df['state_name'].isin(high_gdp_countries)]
df = df.set_index(['state_name', 'year'])
df_high_gdp = df_high_gdp.set_index(['state_name', 'year'])


independent_vars_high_gdp = df_high_gdp[['imf_pr', 'log_gdppc', 'log_fdipc', 'hdi_norm', 'pstab']]

### **Уровень демократии**

In [300]:
dependent_var = df_high_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0148
Estimator:,PanelOLS,R-squared (Between):,0.1893
No. Observations:,600,R-squared (Within):,0.0039
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0148
Time:,17:02:28,Log-likelihood,-1529.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.7841
Entities:,30,P-value,0.1141
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.6115


In [301]:
dependent_var = df_high_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0053
Estimator:,PanelOLS,R-squared (Between):,-3.7711
No. Observations:,600,R-squared (Within):,0.0024
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2196
Time:,17:02:28,Log-likelihood,-1520.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.6125
Entities:,30,P-value,0.6904
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2892


In [302]:
dependent_var = df_high_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0170
Estimator:,PanelOLS,R-squared (Between):,-13.657
No. Observations:,600,R-squared (Within):,0.0170
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7872
Time:,17:02:28,Log-likelihood,-1510.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.9509
Entities:,30,P-value,0.0843
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,4.0578


In [303]:
dependent_var = df_high_gdp['libdem_norm_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0170
Estimator:,PanelOLS,R-squared (Between):,-150.62
No. Observations:,600,R-squared (Within):,0.0072
Date:,"Mon, May 19 2025",R-squared (Overall):,-8.8512
Time:,17:02:28,Log-likelihood,-1500.5
Cov. Estimator:,Robust,,
,,F-statistic:,1.8853
Entities:,30,P-value,0.0951
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,4.1292


### **Защита прав собственности**

In [304]:
dependent_var = df_high_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0278
Estimator:,PanelOLS,R-squared (Between):,0.2509
No. Observations:,600,R-squared (Within):,0.0219
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0278
Time:,17:02:28,Log-likelihood,-1956.7
Cov. Estimator:,Robust,,
,,F-statistic:,3.4016
Entities:,30,P-value,0.0049
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.7427


In [305]:
dependent_var = df_high_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0102
Estimator:,PanelOLS,R-squared (Between):,-7.6270
No. Observations:,600,R-squared (Within):,0.0034
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1921
Time:,17:02:28,Log-likelihood,-1774.8
Cov. Estimator:,Robust,,
,,F-statistic:,1.1806
Entities:,30,P-value,0.3172
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.5584


In [306]:
dependent_var = df_high_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0344
Estimator:,PanelOLS,R-squared (Between):,-56.568
No. Observations:,600,R-squared (Within):,0.0344
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.4153
Time:,17:02:28,Log-likelihood,-1946.9
Cov. Estimator:,Robust,,
,,F-statistic:,4.0308
Entities:,30,P-value,0.0013
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.6084


In [307]:
dependent_var = df_high_gdp['property_rights_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0105
Estimator:,PanelOLS,R-squared (Between):,-85.763
No. Observations:,600,R-squared (Within):,-0.0374
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.2330
Time:,17:02:28,Log-likelihood,-1763.6
Cov. Estimator:,Robust,,
,,F-statistic:,1.1605
Entities:,30,P-value,0.3274
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8137


### **Налоговое бремя**

In [308]:
dependent_var = df_high_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0386
Estimator:,PanelOLS,R-squared (Between):,0.3677
No. Observations:,600,R-squared (Within):,0.0147
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0386
Time:,17:02:28,Log-likelihood,-1440.0
Cov. Estimator:,Robust,,
,,F-statistic:,4.7772
Entities:,30,P-value,0.0003
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.4554


In [309]:
dependent_var = df_high_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0135
Estimator:,PanelOLS,R-squared (Between):,-3.4976
No. Observations:,600,R-squared (Within):,0.0131
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2246
Time:,17:02:28,Log-likelihood,-1431.0
Cov. Estimator:,Robust,,
,,F-statistic:,1.5727
Entities:,30,P-value,0.1658
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.3652


In [310]:
dependent_var = df_high_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0212
Estimator:,PanelOLS,R-squared (Between):,-14.120
No. Observations:,600,R-squared (Within):,0.0212
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.9362
Time:,17:02:28,Log-likelihood,-1424.3
Cov. Estimator:,Robust,,
,,F-statistic:,2.4437
Entities:,30,P-value,0.0332
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.9356


In [311]:
dependent_var = df_high_gdp['tax_burden_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0111
Estimator:,PanelOLS,R-squared (Between):,-49.274
No. Observations:,600,R-squared (Within):,-0.0013
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.3373
Time:,17:02:28,Log-likelihood,-1416.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.2286
Entities:,30,P-value,0.2942
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.9809


### **Государственные расходы**

In [312]:
dependent_var = df_high_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0017
Estimator:,PanelOLS,R-squared (Between):,0.0866
No. Observations:,600,R-squared (Within):,0.0003
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0017
Time:,17:02:28,Log-likelihood,-1965.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.2028
Entities:,30,P-value,0.9613
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2159


In [313]:
dependent_var = df_high_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,-96.910
No. Observations:,600,R-squared (Within):,0.0004
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.5228
Time:,17:02:28,Log-likelihood,-1949.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.3926
Entities:,30,P-value,0.8540
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4898


In [314]:
dependent_var = df_high_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0030
Estimator:,PanelOLS,R-squared (Between):,-45.047
No. Observations:,600,R-squared (Within):,0.0030
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.7051
Time:,17:02:29,Log-likelihood,-1960.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.3369
Entities:,30,P-value,0.8906
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4031


In [315]:
dependent_var = df_high_gdp['government_spending_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0133
Estimator:,PanelOLS,R-squared (Between):,-2140.5
No. Observations:,600,R-squared (Within):,-0.0967
Date:,"Mon, May 19 2025",R-squared (Overall):,-33.739
Time:,17:02:29,Log-likelihood,-1941.9
Cov. Estimator:,Robust,,
,,F-statistic:,1.4672
Entities:,30,P-value,0.1988
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1572


### **Свобода бизнеса**

In [316]:
dependent_var = df_high_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0055
Estimator:,PanelOLS,R-squared (Between):,-0.2450
No. Observations:,600,R-squared (Within):,0.0106
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0055
Time:,17:02:29,Log-likelihood,-1749.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.6633
Entities:,30,P-value,0.6514
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5627


In [317]:
dependent_var = df_high_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0037
Estimator:,PanelOLS,R-squared (Between):,-21.243
No. Observations:,600,R-squared (Within):,-0.0098
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.4316
Time:,17:02:29,Log-likelihood,-1716.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.4254
Entities:,30,P-value,0.8311
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5489


In [318]:
dependent_var = df_high_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0354
Estimator:,PanelOLS,R-squared (Between):,-48.892
No. Observations:,600,R-squared (Within):,0.0354
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.9367
Time:,17:02:29,Log-likelihood,-1733.8
Cov. Estimator:,Robust,,
,,F-statistic:,4.1440
Entities:,30,P-value,0.0010
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,3.4134


In [319]:
dependent_var = df_high_gdp['business_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0197
Estimator:,PanelOLS,R-squared (Between):,-664.06
No. Observations:,600,R-squared (Within):,-0.0440
Date:,"Mon, May 19 2025",R-squared (Overall):,-13.237
Time:,17:02:29,Log-likelihood,-1705.4
Cov. Estimator:,Robust,,
,,F-statistic:,2.1966
Entities:,30,P-value,0.0533
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.9447


### **Монетарная свобода**

In [320]:
dependent_var = df_high_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0150
Estimator:,PanelOLS,R-squared (Between):,0.0762
No. Observations:,600,R-squared (Within):,0.0096
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0150
Time:,17:02:29,Log-likelihood,-1687.3
Cov. Estimator:,Robust,,
,,F-statistic:,1.8170
Entities:,30,P-value,0.1075
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.7283


In [321]:
dependent_var = df_high_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0139
Estimator:,PanelOLS,R-squared (Between):,-35.693
No. Observations:,600,R-squared (Within):,0.0146
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.9224
Time:,17:02:29,Log-likelihood,-1657.4
Cov. Estimator:,Robust,,
,,F-statistic:,1.6224
Entities:,30,P-value,0.1521
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.4355


In [322]:
dependent_var = df_high_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0182
Estimator:,PanelOLS,R-squared (Between):,-39.211
No. Observations:,600,R-squared (Within):,0.0182
Date:,"Mon, May 19 2025",R-squared (Overall):,-3.2084
Time:,17:02:29,Log-likelihood,-1660.6
Cov. Estimator:,Robust,,
,,F-statistic:,2.0915
Entities:,30,P-value,0.0649
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.2506


In [323]:
dependent_var = df_high_gdp['monetary_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0074
Estimator:,PanelOLS,R-squared (Between):,-36.071
No. Observations:,600,R-squared (Within):,0.0171
Date:,"Mon, May 19 2025",R-squared (Overall):,-2.9512
Time:,17:02:29,Log-likelihood,-1631.1
Cov. Estimator:,Robust,,
,,F-statistic:,0.8194
Entities:,30,P-value,0.5361
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0608


### **Свобода торговли**

In [324]:
dependent_var = df_high_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0068
Estimator:,PanelOLS,R-squared (Between):,0.1460
No. Observations:,600,R-squared (Within):,0.0017
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0068
Time:,17:02:29,Log-likelihood,-1920.2
Cov. Estimator:,Robust,,
,,F-statistic:,0.8173
Entities:,30,P-value,0.5375
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1794


In [325]:
dependent_var = df_high_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-9.1972
No. Observations:,600,R-squared (Within):,0.0007
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.3240
Time:,17:02:29,Log-likelihood,-1885.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.6477
Entities:,30,P-value,0.6634
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5135


In [326]:
dependent_var = df_high_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0397
Estimator:,PanelOLS,R-squared (Between):,-611.54
No. Observations:,600,R-squared (Within):,0.0397
Date:,"Mon, May 19 2025",R-squared (Overall):,-21.552
Time:,17:02:29,Log-likelihood,-1899.3
Cov. Estimator:,Robust,,
,,F-statistic:,4.6717
Entities:,30,P-value,0.0003
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.5554


In [327]:
dependent_var = df_high_gdp['investment_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0113
Estimator:,PanelOLS,R-squared (Between):,-53.921
No. Observations:,600,R-squared (Within):,-0.0675
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.9688
Time:,17:02:29,Log-likelihood,-1872.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.2445
Entities:,30,P-value,0.2869
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8323


### **Финансовая свобода**

In [328]:
dependent_var = df_high_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,0.0344
No. Observations:,600,R-squared (Within):,0.0016
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0023
Time:,17:02:29,Log-likelihood,-1890.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.2701
Entities:,30,P-value,0.9295
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2717


In [329]:
dependent_var = df_high_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0014
Estimator:,PanelOLS,R-squared (Between):,-1.8544
No. Observations:,600,R-squared (Within):,0.0008
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0349
Time:,17:02:29,Log-likelihood,-1881.0
Cov. Estimator:,Robust,,
,,F-statistic:,0.1611
Entities:,30,P-value,0.9766
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.1608


In [330]:
dependent_var = df_high_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0078
Estimator:,PanelOLS,R-squared (Between):,-3.9131
No. Observations:,600,R-squared (Within):,0.0078
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0677
Time:,17:02:29,Log-likelihood,-1882.9
Cov. Estimator:,Robust,,
,,F-statistic:,0.8903
Entities:,30,P-value,0.4872
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.4184


In [331]:
dependent_var = df_high_gdp['financial_freedom_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0061
Estimator:,PanelOLS,R-squared (Between):,-2.0954
No. Observations:,600,R-squared (Within):,0.0040
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.0365
Time:,17:02:29,Log-likelihood,-1873.6
Cov. Estimator:,Robust,,
,,F-statistic:,0.6696
Entities:,30,P-value,0.6467
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2498


### **Государственная эффективность**

In [332]:
dependent_var = df_high_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,-0.0369
No. Observations:,600,R-squared (Within):,0.0036
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0023
Time:,17:02:29,Log-likelihood,-1767.4
Cov. Estimator:,Robust,,
,,F-statistic:,0.2746
Entities:,30,P-value,0.9270
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.2411


In [333]:
dependent_var = df_high_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-149.06
No. Observations:,600,R-squared (Within):,-0.0026
Date:,"Mon, May 19 2025",R-squared (Overall):,-4.7062
Time:,17:02:29,Log-likelihood,-1745.8
Cov. Estimator:,Robust,,
,,F-statistic:,0.9980
Entities:,30,P-value,0.4181
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.9949


In [334]:
dependent_var = df_high_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0185
Estimator:,PanelOLS,R-squared (Between):,-58.718
No. Observations:,600,R-squared (Within):,0.0185
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.8349
Time:,17:02:29,Log-likelihood,-1752.9
Cov. Estimator:,Robust,,
,,F-statistic:,2.1340
Entities:,30,P-value,0.0599
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.3255


In [335]:
dependent_var = df_high_gdp['gov_effectiveness_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,-133.31
No. Observations:,600,R-squared (Within):,0.0160
Date:,"Mon, May 19 2025",R-squared (Overall):,-4.1910
Time:,17:02:29,Log-likelihood,-1736.7
Cov. Estimator:,Robust,,
,,F-statistic:,0.4994
Entities:,30,P-value,0.7768
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6504


### **Уровень коррупции**

In [336]:
dependent_var = df_high_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0077
Estimator:,PanelOLS,R-squared (Between):,0.1091
No. Observations:,600,R-squared (Within):,0.0034
Date:,"Mon, May 19 2025",R-squared (Overall):,0.0077
Time:,17:02:29,Log-likelihood,-1670.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.9181
Entities:,30,P-value,0.4687
Avg Obs:,20.000,Distribution:,"F(5,595)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0424


In [337]:
dependent_var = df_high_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=False, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,-5.4527
No. Observations:,600,R-squared (Within):,0.0023
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.2181
Time:,17:02:30,Log-likelihood,-1644.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.3918
Entities:,30,P-value,0.8545
Avg Obs:,20.000,Distribution:,"F(5,575)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5075


In [338]:
dependent_var = df_high_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=False)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0111
Estimator:,PanelOLS,R-squared (Between):,-30.938
No. Observations:,600,R-squared (Within):,0.0111
Date:,"Mon, May 19 2025",R-squared (Overall):,-1.2393
Time:,17:02:30,Log-likelihood,-1657.1
Cov. Estimator:,Robust,,
,,F-statistic:,1.2665
Entities:,30,P-value,0.2769
Avg Obs:,20.000,Distribution:,"F(5,565)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.0000


In [339]:
dependent_var = df_high_gdp['corruption_diff_lagged']

model = PanelOLS(dependent_var, independent_vars_high_gdp, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0064
Estimator:,PanelOLS,R-squared (Between):,-3.9680
No. Observations:,600,R-squared (Within):,0.0049
Date:,"Mon, May 19 2025",R-squared (Overall):,-0.1556
Time:,17:02:30,Log-likelihood,-1630.5
Cov. Estimator:,Robust,,
,,F-statistic:,0.7070
Entities:,30,P-value,0.6183
Avg Obs:,20.000,Distribution:,"F(5,546)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5566


In [340]:
# ['libdem_norm_diff_lagged', 'property_rights_diff_lagged', 'tax_burden_diff_lagged', 'government_spending_diff_lagged', 
# 'business_freedom_diff_lagged', 'monetary_freedom_diff_lagged', 'trade_freedom_diff_lagged', 'investment_freedom_diff_lagged', 
# 'financial_freedom_diff_lagged', 'gov_effectiveness_diff_lagged', 'corruption_diff_lagged']